# Create validation set

In [1]:
path = 'data/fish'
trainpath = path + '/train'
validpath = path + '/valid'
models_path = path+'/models/'

In [2]:
import os
import random

if not os.path.exists(models_path):
    os.makedirs(models_path)

valid_perc = 0.15
for fish_type in os.listdir(trainpath):
    type_train_path = os.path.join(trainpath, fish_type)
    type_valid_path = os.path.join(validpath, fish_type)
    if not os.path.exists(type_valid_path):
        os.makedirs(type_valid_path)
        images = os.listdir(type_train_path)
        valid_images = random.sample(images, int(len(images) * valid_perc))
        for image in valid_images:
            os.rename(os.path.join(type_train_path, image), os.path.join(type_valid_path, image))

# Build the VGG16 Model

In [49]:
from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
import numpy as np

vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3, 1, 1))
def vgg_preprocess(x):
    # subtracts the mean so that we get a 0-centered value
    x = x - vgg_mean
    
    # reverses the axis, since most pretrained data comes from OpenCV, which uses BGR rather than RGB
    return x[:, ::-1]

def vgg_convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model = Sequential()
model.add(Lambda(vgg_preprocess, input_shape=(3, 224, 224), output_shape=(3,224,224)))
vgg_convblock(model, 2, 64)
vgg_convblock(model, 2, 128)
vgg_convblock(model, 3, 256)
vgg_convblock(model, 3, 512)
vgg_convblock(model, 3, 512)

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

download_path = 'http://files.fast.ai/models/'
weights_file = get_file('vgg16.h5', download_path+'vgg16.h5', cache_subdir='models')
model.load_weights(weights_file)

In [50]:
conv_layers = [layer for layer in model.layers if type(layer) in [Convolution2D, MaxPooling2D]]
for layer in conv_layers:
    layer.trainable = False

In [51]:
from keras.layers.normalization import BatchNormalization
model = Sequential([BatchNormalization(axis=1, input_shape=(3, 224, 224))] + conv_layers + [
    BatchNormalization(axis=1),
    Flatten(),
    Dense(4096, activation='relu'),
    BatchNormalization(),
    Dropout(0.25),
    Dense(4096, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(8, activation='softmax')
])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_5 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        lambda_2[0][0]                   
                                                                   batchnormalization_5[0][0]       
____________________________________________________________________________________________________
convolution2d_15 (Convolution2D) (None, 64, 224, 224)  36928       convolution2d_14[0][0]           
                                                                   convolution2d_14[1][0]           
___________________________________________________________________________________________

In [52]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True):
    return ImageDataGenerator().flow_from_directory(path, target_size=(224, 224), batch_size=32, class_mode=class_mode, shuffle=shuffle)

In [54]:
train_gen = batch_gen(trainpath)
valid_gen = batch_gen(validpath)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [12]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 103s - loss: 0.4280 - acc: 0.8852 - val_loss: 0.2646 - val_acc: 0.9183
Epoch 2/4
3214/3214 [==============================] - 96s - loss: 0.2117 - acc: 0.9449 - val_loss: 0.1634 - val_acc: 0.9520
Epoch 3/4
3214/3214 [==============================] - 95s - loss: 0.1351 - acc: 0.9633 - val_loss: 0.1347 - val_acc: 0.9680
Epoch 4/4
3214/3214 [==============================] - 95s - loss: 0.0702 - acc: 0.9779 - val_loss: 0.1697 - val_acc: 0.9627


In [13]:
model.save_weights(models_path+'first_cut_dropout.h5')

In [14]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 101s - loss: 0.0751 - acc: 0.9804 - val_loss: 0.1114 - val_acc: 0.9734
Epoch 2/3
3214/3214 [==============================] - 95s - loss: 0.0483 - acc: 0.9872 - val_loss: 0.0946 - val_acc: 0.9769
Epoch 3/3
3214/3214 [==============================] - 97s - loss: 0.0537 - acc: 0.9872 - val_loss: 0.2305 - val_acc: 0.9645


In [15]:
model.save_weights(models_path+'second_cut_dropout.h5')

In [35]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 116s - loss: 0.2514 - acc: 0.9571 - val_loss: 0.3904 - val_acc: 0.9520
Epoch 2/3
3214/3214 [==============================] - 108s - loss: 0.2511 - acc: 0.9586 - val_loss: 0.3589 - val_acc: 0.9414
Epoch 3/3
3214/3214 [==============================] - 108s - loss: 0.1957 - acc: 0.9689 - val_loss: 0.7567 - val_acc: 0.9076


In [36]:
model.load_weights(models_path+'second_cut.h5')

# Generate a test submission for part 1

In [36]:
testpath = path + '/test_stg1'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)

Found 1000 images belonging to 1 classes.


In [37]:
predictions_one = model.predict_generator(test_gen, test_gen.nb_sample)

In [38]:
import pandas as pd

classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_one = pd.DataFrame(predictions_one, columns=classes)
submission_one.insert(0, 'image', [os.path.basename(filename) for filename in test_gen.filenames])
submission_one.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,1.379791e-11,5.973196e-07,3.163154e-07,6.828305e-06,9.999923e-01,4.275206e-09,6.931159e-08,8.481951e-09
1,img_00007.jpg,9.999681e-01,4.622759e-06,7.769816e-06,1.071996e-05,9.783965e-08,7.005393e-06,1.695339e-06,5.674136e-08
2,img_00009.jpg,9.993801e-01,4.911573e-04,6.267855e-05,5.240328e-06,4.571746e-06,4.364665e-05,1.058294e-05,1.979297e-06
3,img_00018.jpg,9.999210e-01,3.250346e-06,6.027666e-06,4.965854e-07,1.026195e-07,3.665816e-05,2.063032e-05,1.176218e-05
4,img_00027.jpg,4.664029e-02,5.491795e-01,1.300267e-03,1.486103e-03,1.563983e-03,2.944008e-01,1.618629e-02,8.924289e-02


In [39]:
# submission_one.to_csv(os.path.join(path, 'stage_one.csv.gz'), index=False, compression='gzip')

# Generate a test submission for part 2

In [40]:
testpath = path + '/test_stg2'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)

Found 12153 images belonging to 1 classes.


In [41]:
predictions_two = model.predict_generator(test_gen, test_gen.nb_sample)

In [42]:
import pandas as pd

classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_two = pd.DataFrame(predictions_two, columns=classes)
submission_two.insert(0, 'image', ['test_stg2/' + os.path.basename(filename) for filename in test_gen.filenames])
submission_two.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_00001.jpg,0.369125,2.196758e-02,5.095143e-04,0.039584,0.083023,4.181690e-01,1.285864e-04,0.067494
1,test_stg2/image_00002.jpg,0.002792,5.763921e-02,1.185645e-02,0.049786,0.815139,2.100842e-02,2.963617e-03,0.038815
2,test_stg2/image_00003.jpg,0.999793,8.113956e-07,9.289097e-07,0.000045,0.000141,1.278712e-07,4.081627e-07,0.000018
3,test_stg2/image_00004.jpg,0.010733,6.074815e-03,8.940433e-03,0.051236,0.917342,5.015638e-03,1.849269e-04,0.000473
4,test_stg2/image_00005.jpg,0.996308,1.009451e-04,1.893483e-04,0.000175,0.003056,1.078976e-05,9.178784e-06,0.000150


In [45]:
submission = submission_one.append(submission_two, ignore_index=True)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,1.379791e-11,5.973196e-07,3.163154e-07,6.828305e-06,9.999923e-01,4.275206e-09,6.931159e-08,8.481951e-09
1,img_00007.jpg,9.999681e-01,4.622759e-06,7.769816e-06,1.071996e-05,9.783965e-08,7.005393e-06,1.695339e-06,5.674136e-08
2,img_00009.jpg,9.993801e-01,4.911573e-04,6.267855e-05,5.240328e-06,4.571746e-06,4.364665e-05,1.058294e-05,1.979297e-06
3,img_00018.jpg,9.999210e-01,3.250346e-06,6.027666e-06,4.965854e-07,1.026195e-07,3.665816e-05,2.063032e-05,1.176218e-05
4,img_00027.jpg,4.664029e-02,5.491795e-01,1.300267e-03,1.486103e-03,1.563983e-03,2.944008e-01,1.618629e-02,8.924289e-02


In [46]:
submission.to_csv(os.path.join(path, 'stage_two.csv.gz'), index=False, compression='gzip')

At this point, we have a 3.7 private score. ( at the bottom :( )

# Pseudo Labeling

## Construct the first pseudo-labeled set

In [22]:
sample_preds = submission_two.sample(frac=0.15)

In [23]:
pseudotrainpath = path + '/pseudotrain'
if not os.path.exists(pseudotrainpath):
    os.makedirs(pseudotrainpath)

for pred in sample_preds.itertuples(index=False):
    print(sample_preds.columns[np.argmax(pred[1:]) + 1])

BET
YFT
NoF
BET
NoF
OTHER
OTHER
NoF
SHARK
NoF
BET
OTHER
BET
ALB
ALB
BET
NoF
ALB
ALB
NoF
YFT
BET
NoF
BET
NoF
BET
BET
BET
NoF
NoF
YFT
NoF
ALB
NoF
ALB
YFT
NoF
NoF
YFT
BET
ALB
OTHER
NoF
ALB
BET
BET
NoF
ALB
ALB
ALB
BET
NoF
NoF
LAG
NoF
ALB
ALB
NoF
YFT
NoF
BET
OTHER
BET
NoF
BET
NoF
OTHER
NoF
ALB
ALB
ALB
BET
ALB
ALB
NoF
ALB
YFT
NoF
ALB
ALB
OTHER
ALB
ALB
ALB
YFT
ALB
ALB
NoF
NoF
BET
BET
ALB
ALB
LAG
ALB
BET
NoF
YFT
OTHER
ALB
SHARK
BET
NoF
OTHER
OTHER
BET
YFT
NoF
ALB
ALB
OTHER
OTHER
ALB
OTHER
ALB
BET
ALB
YFT
YFT
NoF
BET
BET
BET
BET
YFT
NoF
BET
NoF
BET
NoF
BET
BET
YFT
OTHER
ALB
NoF
OTHER
BET
YFT
YFT
NoF
ALB
BET
OTHER
NoF
OTHER
BET
NoF
ALB
ALB
YFT
OTHER
ALB
YFT
ALB
YFT
ALB
ALB
YFT
OTHER
OTHER
ALB
ALB
DOL
BET
ALB
BET
BET
BET
ALB
NoF
BET
YFT
BET
OTHER
NoF
BET
NoF
NoF
NoF
NoF
NoF
ALB
YFT
YFT
ALB
ALB
SHARK
NoF
ALB
YFT
ALB
YFT
ALB
OTHER
ALB
OTHER
BET
ALB
BET
BET
ALB
BET
ALB
ALB
BET
NoF
BET
ALB
YFT
YFT
YFT
ALB
NoF
NoF
ALB
NoF
YFT
YFT
BET
YFT
ALB
NoF
NoF
ALB
ALB
ALB
YFT
YFT
NoF
NoF
BET
YFT
YFT
ALB
YFT
ALB


Hmm, looks like it's only predicting 3 categories. Let's try some data augmentation first.

# Data augmentation

In [55]:
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0.):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(224, 224), batch_size=48, class_mode=class_mode, shuffle=shuffle)

def image_aug(path):
    return batch_gen(path, zoom_range=0.15, channel_shift_range=0.15, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05)

In [56]:
train_gen = image_aug(trainpath)
#model.load_weights(models_path+'second_cut_dropout.h5')

Found 3214 images belonging to 8 classes.


In [35]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 107s - loss: 0.8957 - acc: 0.8118 - val_loss: 0.2615 - val_acc: 0.9556
Epoch 2/3
3214/3214 [==============================] - 97s - loss: 0.5084 - acc: 0.8696 - val_loss: 0.1718 - val_acc: 0.9716
Epoch 3/3
3214/3214 [==============================] - 98s - loss: 0.3805 - acc: 0.8995 - val_loss: 0.3617 - val_acc: 0.9361


In [47]:
model.save_weights(models_path+'second_cut_data_aug.h5')

In [57]:
model.optimizer.lr = 0.00003
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 105s - loss: 1.9706 - acc: 0.4499 - val_loss: 1.0610 - val_acc: 0.7087
Epoch 2/3
3214/3214 [==============================] - 95s - loss: 0.9041 - acc: 0.7362 - val_loss: 0.5288 - val_acc: 0.8757
Epoch 3/3
3214/3214 [==============================] - 92s - loss: 0.6179 - acc: 0.8158 - val_loss: 0.3274 - val_acc: 0.9165


In [58]:
model.save_weights(models_path+'less_dropout.h5')

# Custom Model

In [17]:
import os
import random
import shutil

samplepath = path + '/sample'

sample_perc = 0.10
for fish_type in os.listdir(trainpath):
    type_train_path = os.path.join(trainpath, fish_type)
    type_sample_path = os.path.join(samplepath, fish_type)
    if not os.path.exists(type_sample_path):
        os.makedirs(type_sample_path)
        images = os.listdir(type_train_path)
        sample_images = random.sample(images, int(len(images) * sample_perc))
        for image in sample_images:
            shutil.copyfile(os.path.join(type_train_path, image), os.path.join(type_sample_path, image))

In [6]:
from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential([BatchNormalization(axis=1, input_shape=(3, 1280, 720))])
convblock(model, 2, 64)
convblock(model, 2, 128)
convblock(model, 3, 256)
convblock(model, 3, 512)
convblock(model, 3, 512)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_31 (BatchNorm (None, 3, 1280, 720)  12          batchnormalization_input_6[0][0] 
____________________________________________________________________________________________________
convolution2d_66 (Convolution2D) (None, 64, 1280, 720) 1792        batchnormalization_31[0][0]      
____________________________________________________________________________________________________
convolution2d_67 (Convolution2D) (None, 64, 1280, 720) 36928       convolution2d_66[0][0]           
____________________________________________________________________________________________________
maxpooling2d_26 (MaxPooling2D)   (None, 64, 640, 360)  0           convolution2d_67[0][0]           
___________________________________________________________________________________________

In [7]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0.):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(1280, 720), batch_size=1, class_mode=class_mode, shuffle=shuffle)

def image_aug(path):
    return batch_gen(path, zoom_range=0.15, channel_shift_range=0.15, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05)

In [14]:
samplepath = path + '/sample'
train_gen = image_aug(trainpath)
valid_gen = batch_gen(validpath)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [15]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=1, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/1
3214/3214 [==============================] - 2776s - loss: 1.6948 - acc: 0.4294 - val_loss: 14.5298 - val_acc: 0.0977


In [16]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 2775s - loss: 1.6180 - acc: 0.4549 - val_loss: 14.8043 - val_acc: 0.0799
Epoch 2/4
3214/3214 [==============================] - 2771s - loss: 1.6179 - acc: 0.4549 - val_loss: 14.9876 - val_acc: 0.0693
Epoch 3/4
3214/3214 [==============================] - 2772s - loss: 1.6178 - acc: 0.4549 - val_loss: 14.9632 - val_acc: 0.0710
Epoch 4/4
3214/3214 [==============================] - 2774s - loss: 1.6178 - acc: 0.4549 - val_loss: 15.0875 - val_acc: 0.0639


In [18]:
model.save_weights(path+ '/models/fat_model.h5')

## Simpler architecture

In [7]:
path = 'data/fish'
trainpath = path + '/train'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 640, 360)))
convblock(model, 1, 64)
convblock(model, 1, 64)
convblock(model, 1, 64)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
    #Dense(4096, activation='relu'),
model.add(Dense(8, activation='softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_5 (BatchNorma (None, 3, 640, 360)   12          batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
convolution2d_4 (Convolution2D)  (None, 64, 640, 360)  1792        batchnormalization_5[0][0]       
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 64, 320, 180)  0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
batchnormalization_6 (BatchNorma (None, 64, 320, 180)  256         maxpooling2d_4[0][0]             
___________________________________________________________________________________________

In [8]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0.):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(640, 360), batch_size=16, class_mode=class_mode, shuffle=shuffle)

def image_aug(path):
    return batch_gen(path, zoom_range=0.15, channel_shift_range=0.15, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05)

In [9]:
train_gen = image_aug(trainpath)
valid_gen = batch_gen(validpath)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [10]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 204s - loss: 1.4913 - acc: 0.5361 - val_loss: 1.4117 - val_acc: 0.5364
Epoch 2/4
3214/3214 [==============================] - 199s - loss: 1.0146 - acc: 0.6686 - val_loss: 0.9203 - val_acc: 0.6323
Epoch 3/4
3214/3214 [==============================] - 199s - loss: 0.8507 - acc: 0.7119 - val_loss: 0.7769 - val_acc: 0.7407
Epoch 4/4
3214/3214 [==============================] - 197s - loss: 0.6812 - acc: 0.7791 - val_loss: 0.5740 - val_acc: 0.8117


In [11]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 203s - loss: 0.5991 - acc: 0.8077 - val_loss: 0.6049 - val_acc: 0.8028
Epoch 2/4
3214/3214 [==============================] - 195s - loss: 0.5467 - acc: 0.8223 - val_loss: 0.4095 - val_acc: 0.8703
Epoch 3/4
3214/3214 [==============================] - 197s - loss: 0.4885 - acc: 0.8388 - val_loss: 0.4001 - val_acc: 0.8508
Epoch 4/4
3214/3214 [==============================] - 199s - loss: 0.4282 - acc: 0.8516 - val_loss: 0.4098 - val_acc: 0.8597


In [12]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 204s - loss: 0.4151 - acc: 0.8684 - val_loss: 0.4388 - val_acc: 0.8845
Epoch 2/4
3214/3214 [==============================] - 196s - loss: 0.3871 - acc: 0.8787 - val_loss: 0.3223 - val_acc: 0.9201
Epoch 3/4
3214/3214 [==============================] - 196s - loss: 0.3542 - acc: 0.8799 - val_loss: 0.3897 - val_acc: 0.8899
Epoch 4/4
3214/3214 [==============================] - 198s - loss: 0.3003 - acc: 0.9042 - val_loss: 0.3080 - val_acc: 0.9094


In [13]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 203s - loss: 0.2935 - acc: 0.9057 - val_loss: 0.3086 - val_acc: 0.9130
Epoch 2/4
3214/3214 [==============================] - 196s - loss: 0.2835 - acc: 0.9060 - val_loss: 0.3348 - val_acc: 0.9023
Epoch 3/4
3214/3214 [==============================] - 195s - loss: 0.2667 - acc: 0.9070 - val_loss: 0.3532 - val_acc: 0.8970
Epoch 4/4
3214/3214 [==============================] - 196s - loss: 0.2486 - acc: 0.9241 - val_loss: 0.2686 - val_acc: 0.9254


In [14]:
model.save_weights(path+'/models/custom_conv.h5')

In [15]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 205s - loss: 0.2402 - acc: 0.9250 - val_loss: 0.2573 - val_acc: 0.9343
Epoch 2/4
3214/3214 [==============================] - 204s - loss: 0.2289 - acc: 0.9294 - val_loss: 0.2403 - val_acc: 0.9325
Epoch 3/4
3214/3214 [==============================] - 199s - loss: 0.2154 - acc: 0.9319 - val_loss: 0.2913 - val_acc: 0.9201
Epoch 4/4
3214/3214 [==============================] - 196s - loss: 0.2057 - acc: 0.9396 - val_loss: 0.2569 - val_acc: 0.9290


In [16]:
model.save_weights(path+'/models/custom_conv2.h5')

## Prepare a submission

In [18]:
testpath = path + '/test_stg1'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)
predictions_one = model.predict_generator(test_gen, test_gen.nb_sample)

import pandas as pd
import os

classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_one = pd.DataFrame(predictions_one, columns=classes)
submission_one.insert(0, 'image', [os.path.basename(filename) for filename in test_gen.filenames])
submission_one.head()

Found 1000 images belonging to 1 classes.


,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.000006,0.000032,0.000110,2.153253e-09,0.999852,1.248368e-07,7.339363e-09,2.623337e-09
1,img_00007.jpg,0.975450,0.000025,0.000003,2.238818e-02,0.000423,5.866907e-04,1.409972e-06,1.122610e-03
2,img_00009.jpg,0.934920,0.004835,0.000010,3.743373e-03,0.011239,2.678013e-02,1.637661e-05,1.845643e-02
3,img_00018.jpg,0.798905,0.000083,0.000132,9.136594e-03,0.000030,4.838634e-02,1.631585e-04,1.431641e-01
4,img_00027.jpg,0.099715,0.181564,0.000827,7.242331e-03,0.094296,4.663059e-03,3.152917e-02,5.801634e-01


In [19]:
testpath = path + '/test_stg2'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)
predictions_two = model.predict_generator(test_gen, test_gen.nb_sample)

classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_two = pd.DataFrame(predictions_two, columns=classes)
submission_two.insert(0, 'image', ['test_stg2/' + os.path.basename(filename) for filename in test_gen.filenames])
submission_two.head()

Found 12153 images belonging to 1 classes.


,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_00001.jpg,8.142726e-11,0.999314,7.991391e-10,1.635908e-05,5.868961e-10,4.200497e-04,1.426409e-04,1.065265e-04
1,test_stg2/image_00002.jpg,7.929316e-02,0.000132,6.366926e-05,1.360887e-07,9.205092e-01,6.528909e-08,5.159534e-08,1.692773e-06
2,test_stg2/image_00003.jpg,1.259260e-03,0.000012,3.848965e-07,1.281208e-08,9.987276e-01,6.673592e-07,1.603590e-08,5.866092e-08
3,test_stg2/image_00004.jpg,2.659394e-01,0.038392,6.429963e-05,8.403962e-03,1.494113e-01,9.197830e-07,1.901453e-05,5.377690e-01
4,test_stg2/image_00005.jpg,2.939177e-02,0.001387,2.137676e-05,2.333543e-05,9.684479e-01,7.837817e-07,2.958562e-06,7.244359e-04


In [20]:
submission = submission_one.append(submission_two, ignore_index=True)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.000006,0.000032,0.000110,2.153253e-09,0.999852,1.248368e-07,7.339363e-09,2.623337e-09
1,img_00007.jpg,0.975450,0.000025,0.000003,2.238818e-02,0.000423,5.866907e-04,1.409972e-06,1.122610e-03
2,img_00009.jpg,0.934920,0.004835,0.000010,3.743373e-03,0.011239,2.678013e-02,1.637661e-05,1.845643e-02
3,img_00018.jpg,0.798905,0.000083,0.000132,9.136594e-03,0.000030,4.838634e-02,1.631585e-04,1.431641e-01
4,img_00027.jpg,0.099715,0.181564,0.000827,7.242331e-03,0.094296,4.663059e-03,3.152917e-02,5.801634e-01


In [21]:
submission.to_csv(os.path.join(path, 'stage_two_custom.csv.gz'), index=False, compression='gzip')

Better, but still not great. Let's try a slightly different architecture.

## Custom 2

In [1]:
path = 'data/fish'
trainpath = path + '/train'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 1280, 720)))
convblock(model, 2, 64)
convblock(model, 2, 64)
convblock(model, 2, 64)
convblock(model, 2, 64)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))
model.summary()

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 Ti (CNMeM is disabled, cuDNN 5110)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 1280, 720)  12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 1280, 720) 1792        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 1280, 720) 36928       convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 640, 360)  0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [2]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0.):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(1280, 720), batch_size=4, class_mode=class_mode, shuffle=shuffle)

def image_aug(path):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05)

In [3]:
train_gen = image_aug(trainpath)
valid_gen = batch_gen(validpath)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [4]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 900s - loss: 1.2481 - acc: 0.5840 - val_loss: 0.8406 - val_acc: 0.7105
Epoch 2/4
3214/3214 [==============================] - 897s - loss: 0.8450 - acc: 0.7110 - val_loss: 0.5504 - val_acc: 0.8153
Epoch 3/4
3214/3214 [==============================] - 900s - loss: 0.6439 - acc: 0.7729 - val_loss: 0.5937 - val_acc: 0.7904
Epoch 4/4
3214/3214 [==============================] - 898s - loss: 0.5197 - acc: 0.8298 - val_loss: 0.3731 - val_acc: 0.8544


In [5]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 897s - loss: 0.4287 - acc: 0.8597 - val_loss: 0.3443 - val_acc: 0.8774
Epoch 2/6
3214/3214 [==============================] - 897s - loss: 0.3650 - acc: 0.8759 - val_loss: 0.4177 - val_acc: 0.8774
Epoch 3/6
3214/3214 [==============================] - 897s - loss: 0.3297 - acc: 0.8917 - val_loss: 0.3506 - val_acc: 0.9023
Epoch 4/6
3214/3214 [==============================] - 900s - loss: 0.3097 - acc: 0.8992 - val_loss: 0.3008 - val_acc: 0.9076
Epoch 5/6
3214/3214 [==============================] - 899s - loss: 0.2706 - acc: 0.9113 - val_loss: 0.3262 - val_acc: 0.9059
Epoch 6/6
3214/3214 [==============================] - 896s - loss: 0.2308 - acc: 0.9238 - val_loss: 0.2620 - val_acc: 0.9183


In [6]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 899s - loss: 0.2167 - acc: 0.9309 - val_loss: 0.2284 - val_acc: 0.9325
Epoch 2/4
3214/3214 [==============================] - 898s - loss: 0.2034 - acc: 0.9337 - val_loss: 0.2131 - val_acc: 0.9343
Epoch 3/4
3214/3214 [==============================] - 897s - loss: 0.1876 - acc: 0.9356 - val_loss: 0.2833 - val_acc: 0.9290
Epoch 4/4
3214/3214 [==============================] - 897s - loss: 0.1807 - acc: 0.9421 - val_loss: 0.1812 - val_acc: 0.9556


In [7]:
model.save_weights(path + '/models/custom_larger.h5')

In [12]:
testpath = path + '/test_stg1'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)

Found 1000 images belonging to 1 classes.


In [ ]:
predictions_one = model.predict_generator(test_gen, test_gen.nb_sample)

In [13]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/8, mx)

import pandas as pd
import os

classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_one = pd.DataFrame(do_clip(predictions_one, 0.92), columns=classes)
submission_one.insert(0, 'image', [os.path.basename(filename) for filename in test_gen.filenames])
submission_one.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.010000,0.01,0.01,0.010000,0.92,0.010000,0.01,0.010000
1,img_00007.jpg,0.920000,0.01,0.01,0.010861,0.01,0.010000,0.01,0.010000
2,img_00009.jpg,0.920000,0.01,0.01,0.010000,0.01,0.017057,0.01,0.010000
3,img_00018.jpg,0.920000,0.01,0.01,0.010000,0.01,0.010000,0.01,0.014938
4,img_00027.jpg,0.888394,0.01,0.01,0.010000,0.01,0.010000,0.01,0.109281


In [14]:
testpath = path + '/test_stg2'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)

Found 12153 images belonging to 1 classes.


In [ ]:
predictions_two = model.predict_generator(test_gen, test_gen.nb_sample)

In [15]:
classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_two = pd.DataFrame(do_clip(predictions_two, 0.92), columns=classes)
submission_two.insert(0, 'image', ['test_stg2/' + os.path.basename(filename) for filename in test_gen.filenames])
submission_two.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_00001.jpg,0.010000,0.485560,0.01,0.010000,0.496071,0.01,0.01,0.011096
1,test_stg2/image_00002.jpg,0.010000,0.010000,0.01,0.010000,0.920000,0.01,0.01,0.010000
2,test_stg2/image_00003.jpg,0.035360,0.010000,0.01,0.010000,0.920000,0.01,0.01,0.010000
3,test_stg2/image_00004.jpg,0.696230,0.046385,0.01,0.083992,0.046925,0.01,0.01,0.126347
4,test_stg2/image_00005.jpg,0.226159,0.010000,0.01,0.010000,0.766131,0.01,0.01,0.010000


In [16]:
submission = submission_one.append(submission_two, ignore_index=True)
submission.to_csv(os.path.join(path, 'stage_two_custom_larger_clipping.csv.gz'), index=False, compression='gzip')

# Custom 3

In [1]:
path = 'data/fish'
trainpath = path + '/train'
samplepath = path + '/sample'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 1280, 720)))
convblock(model, 1, 64)
convblock(model, 1, 128)
convblock(model, 1, 256)
convblock(model, 1, 128)
convblock(model, 1, 64)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))
model.summary()

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 Ti (CNMeM is disabled, cuDNN 5110)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 1280, 720)  12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 1280, 720) 1792        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 640, 360)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 64, 640, 360)  256         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [2]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(1280, 720), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05)

In [3]:
train_gen = image_aug(trainpath)
valid_gen = batch_gen(validpath, batch_size=16)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [4]:
#model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=2, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/2
3214/3214 [==============================] - 710s - loss: 2.0914 - acc: 0.3258 - val_loss: 1.6697 - val_acc: 0.4813
Epoch 2/2
3214/3214 [==============================] - 706s - loss: 1.7590 - acc: 0.4147 - val_loss: 1.4700 - val_acc: 0.5240


In [ ]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 712s - loss: 1.6036 - acc: 0.4757 - val_loss: 1.3454 - val_acc: 0.6039
Epoch 2/6
3214/3214 [==============================] - 712s - loss: 1.5337 - acc: 0.5050 - val_loss: 1.2101 - val_acc: 0.6838
Epoch 3/6
3214/3214 [==============================] - 702s - loss: 1.4715 - acc: 0.5134 - val_loss: 1.1802 - val_acc: 0.6590
Epoch 4/6
3214/3214 [==============================] - 702s - loss: 1.3839 - acc: 0.5600 - val_loss: 1.2984 - val_acc: 0.6803
Epoch 5/6
3214/3214 [==============================] - 704s - loss: 1.3426 - acc: 0.5790 - val_loss: 1.1486 - val_acc: 0.6163
Epoch 6/6
3214/3214 [==============================] - 700s - loss: 1.3152 - acc: 0.5831 - val_loss: 1.0705 - val_acc: 0.6803


In [ ]:
model.save_weights(modelpath+'/custom3.h5')

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 700s - loss: 1.2409 - acc: 0.6017 - val_loss: 0.9413 - val_acc: 0.6963
Epoch 2/6
3214/3214 [==============================] - 698s - loss: 1.1895 - acc: 0.6322 - val_loss: 0.8372 - val_acc: 0.7549
Epoch 3/6
3214/3214 [==============================] - 700s - loss: 1.1725 - acc: 0.6332 - val_loss: 0.9466 - val_acc: 0.7531
Epoch 4/6
3214/3214 [==============================] - 702s - loss: 1.1527 - acc: 0.6413 - val_loss: 0.8056 - val_acc: 0.7673
Epoch 5/6
2048/3214 [==================>...........] - ETA: 220s - loss: 1.0884 - acc: 0.6572

In [ ]:
model.save_weights(modelpath+'/custom3.h5')

# Simpler arch 2

In [12]:
path = 'data/fish'
trainpath = path + '/train'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 640, 360)))
convblock(model, 1, 64)
model.add(Dropout(0.1))
convblock(model, 1, 64)
model.add(Dropout(0.1))
convblock(model, 1, 64)
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_13 (BatchNorm (None, 3, 640, 360)   12          batchnormalization_input_4[0][0] 
____________________________________________________________________________________________________
convolution2d_10 (Convolution2D) (None, 64, 640, 360)  1792        batchnormalization_13[0][0]      
____________________________________________________________________________________________________
maxpooling2d_10 (MaxPooling2D)   (None, 64, 320, 180)  0           convolution2d_10[0][0]           
____________________________________________________________________________________________________
batchnormalization_14 (BatchNorm (None, 64, 320, 180)  256         maxpooling2d_10[0][0]            
___________________________________________________________________________________________

In [13]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(640, 360), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size)

In [14]:
train_gen = image_aug(trainpath, batch_size=16)
valid_gen = batch_gen(validpath, batch_size=32)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [15]:
#model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=2, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/2
3214/3214 [==============================] - 215s - loss: 2.0610 - acc: 0.4919 - val_loss: 2.5846 - val_acc: 0.4565
Epoch 2/2
3214/3214 [==============================] - 211s - loss: 1.4702 - acc: 0.5865 - val_loss: 1.4680 - val_acc: 0.5417


In [17]:
model.save_weights(path+'/models/simpler2.h5')

In [18]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 221s - loss: 1.2986 - acc: 0.6232 - val_loss: 0.9922 - val_acc: 0.6909
Epoch 2/6
3214/3214 [==============================] - 210s - loss: 1.1334 - acc: 0.6649 - val_loss: 0.9085 - val_acc: 0.6643
Epoch 3/6
3214/3214 [==============================] - 211s - loss: 1.0204 - acc: 0.6886 - val_loss: 0.9947 - val_acc: 0.7194
Epoch 4/6
3214/3214 [==============================] - 214s - loss: 0.9301 - acc: 0.7038 - val_loss: 0.7938 - val_acc: 0.7318
Epoch 5/6
3214/3214 [==============================] - 211s - loss: 0.8505 - acc: 0.7324 - val_loss: 0.7846 - val_acc: 0.7460
Epoch 6/6
3214/3214 [==============================] - 210s - loss: 0.7990 - acc: 0.7365 - val_loss: 0.8452 - val_acc: 0.7425


In [19]:
model.save_weights(path+'/models/simpler2_2.h5')

In [20]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 214s - loss: 0.7373 - acc: 0.7598 - val_loss: 0.9518 - val_acc: 0.7034
Epoch 2/6
3214/3214 [==============================] - 207s - loss: 0.7495 - acc: 0.7710 - val_loss: 0.7078 - val_acc: 0.7833
Epoch 3/6
3214/3214 [==============================] - 210s - loss: 0.7240 - acc: 0.7741 - val_loss: 0.9394 - val_acc: 0.7140
Epoch 4/6
3214/3214 [==============================] - 209s - loss: 0.6510 - acc: 0.7931 - val_loss: 0.7124 - val_acc: 0.7620
Epoch 5/6
3214/3214 [==============================] - 208s - loss: 0.6503 - acc: 0.8015 - val_loss: 0.8321 - val_acc: 0.7425
Epoch 6/6
3214/3214 [==============================] - 206s - loss: 0.6082 - acc: 0.8074 - val_loss: 0.7398 - val_acc: 0.7584


In [21]:
model.save_weights(path+'/models/simpler2_3.h5')

In [22]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 219s - loss: 0.6113 - acc: 0.8111 - val_loss: 0.7454 - val_acc: 0.7496
Epoch 2/6
2240/3214 [===================>..........] - ETA: 53s - loss: 0.5733 - acc: 0.8152

KeyboardInterrupt: 

# Simpler architecture again

In [2]:
path = 'data/fish'
trainpath = path + '/train'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 640, 360)))
convblock(model, 1, 64)
convblock(model, 1, 128)
convblock(model, 1, 128)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
    #Dense(4096, activation='relu'),
model.add(Dense(8, activation='softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 640, 360)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 640, 360)  1792        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 320, 180)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 64, 320, 180)  256         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [3]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(640, 360), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=180)

In [4]:
train_gen = image_aug(trainpath, batch_size=16)
valid_gen = batch_gen(validpath, batch_size=32)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [5]:
#model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=2, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/2
3214/3214 [==============================] - 219s - loss: 1.7348 - acc: 0.4564 - val_loss: 1.9861 - val_acc: 0.3357
Epoch 2/2
3214/3214 [==============================] - 216s - loss: 1.3947 - acc: 0.5314 - val_loss: 1.4687 - val_acc: 0.5062


In [6]:
model.save_weights(path + '/models/simpler3.h5')

In [7]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 224s - loss: 1.2669 - acc: 0.5635 - val_loss: 1.3074 - val_acc: 0.5204
Epoch 2/6
3214/3214 [==============================] - 209s - loss: 1.2479 - acc: 0.5713 - val_loss: 1.1722 - val_acc: 0.5631
Epoch 3/6
3214/3214 [==============================] - 209s - loss: 1.1467 - acc: 0.6033 - val_loss: 1.1465 - val_acc: 0.5222
Epoch 4/6
3214/3214 [==============================] - 210s - loss: 1.0891 - acc: 0.6207 - val_loss: 1.1124 - val_acc: 0.6039
Epoch 5/6
3214/3214 [==============================] - 214s - loss: 1.0654 - acc: 0.6232 - val_loss: 1.0111 - val_acc: 0.6217
Epoch 6/6
3214/3214 [==============================] - 212s - loss: 1.0423 - acc: 0.6431 - val_loss: 0.9338 - val_acc: 0.6483


In [8]:
model.save_weights(path + '/models/simpler3_2.h5')

In [9]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 219s - loss: 0.9596 - acc: 0.6559 - val_loss: 0.8926 - val_acc: 0.6714
Epoch 2/6
3214/3214 [==============================] - 208s - loss: 0.9693 - acc: 0.6500 - val_loss: 0.8692 - val_acc: 0.7052
Epoch 3/6
3214/3214 [==============================] - 212s - loss: 0.9343 - acc: 0.6717 - val_loss: 0.9887 - val_acc: 0.6874
Epoch 4/6
3214/3214 [==============================] - 213s - loss: 0.9100 - acc: 0.6752 - val_loss: 0.9572 - val_acc: 0.6483
Epoch 5/6
3214/3214 [==============================] - 207s - loss: 0.8910 - acc: 0.6758 - val_loss: 0.8896 - val_acc: 0.6856
Epoch 6/6
3214/3214 [==============================] - 210s - loss: 0.8653 - acc: 0.6845 - val_loss: 0.8325 - val_acc: 0.6785


In [10]:
model.save_weights(path + '/models/simpler3_3.h5')

In [11]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=2, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/2
 896/3214 [=======>......................] - ETA: 138s - loss: 0.8881 - acc: 0.6775

KeyboardInterrupt: 

# Another model

In [1]:
path = 'data/fish'
trainpath = path + '/train'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 640, 360)))
convblock(model, 1, 128)
convblock(model, 1, 128)
convblock(model, 1, 128)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))
model.summary()

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 Ti (CNMeM is disabled, cuDNN 5110)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 640, 360)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 128, 640, 360) 3584        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 128, 320, 180) 0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 128, 320, 180) 512         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [2]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(640, 360), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=180)

In [3]:
train_gen = image_aug(trainpath, batch_size=16)
valid_gen = batch_gen(validpath, batch_size=32)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [4]:
#model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=2, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/2
3214/3214 [==============================] - 247s - loss: 2.0024 - acc: 0.4428 - val_loss: 2.1852 - val_acc: 0.3694
Epoch 2/2
3214/3214 [==============================] - 236s - loss: 1.5298 - acc: 0.5227 - val_loss: 1.7632 - val_acc: 0.4352


In [5]:
model.save_weights(path+'/models/simpler128.h5')

In [6]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 248s - loss: 1.3212 - acc: 0.5719 - val_loss: 1.3673 - val_acc: 0.5293
Epoch 2/4
3214/3214 [==============================] - 244s - loss: 1.2056 - acc: 0.5881 - val_loss: 1.2215 - val_acc: 0.5897
Epoch 3/4
3214/3214 [==============================] - 237s - loss: 1.1686 - acc: 0.6083 - val_loss: 1.3588 - val_acc: 0.6181
Epoch 4/4
3214/3214 [==============================] - 239s - loss: 1.0760 - acc: 0.6266 - val_loss: 1.3255 - val_acc: 0.5631


In [8]:
model.save_weights(path+'/models/simpler128_1.h5')

In [9]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 250s - loss: 1.0628 - acc: 0.6332 - val_loss: 1.0638 - val_acc: 0.6270
Epoch 2/4
3214/3214 [==============================] - 242s - loss: 1.0374 - acc: 0.6413 - val_loss: 0.9757 - val_acc: 0.6821
Epoch 3/4
3214/3214 [==============================] - 244s - loss: 0.9774 - acc: 0.6668 - val_loss: 1.0688 - val_acc: 0.6767
Epoch 4/4
3214/3214 [==============================] - 245s - loss: 0.9655 - acc: 0.6705 - val_loss: 0.8729 - val_acc: 0.6856


In [10]:
model.save_weights(path+'/models/simpler128_2.h5')

In [11]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 251s - loss: 0.9493 - acc: 0.6770 - val_loss: 0.8875 - val_acc: 0.7211
Epoch 2/4
3214/3214 [==============================] - 243s - loss: 0.9019 - acc: 0.6861 - val_loss: 0.9552 - val_acc: 0.6554
Epoch 3/4
3214/3214 [==============================] - 259s - loss: 0.8380 - acc: 0.6966 - val_loss: 0.9574 - val_acc: 0.7069
Epoch 4/4
3214/3214 [==============================] - 257s - loss: 0.8668 - acc: 0.6979 - val_loss: 0.7748 - val_acc: 0.7353


In [12]:
model.save_weights(path+'/models/simpler128_3.h5')

# Simpler again

In [1]:
path = 'data/fish'
trainpath = path + '/train'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 640, 360)))
convblock(model, 1, 64)
convblock(model, 1, 64)
convblock(model, 1, 64)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
    #Dense(4096, activation='relu'),
model.add(Dense(8, activation='softmax'))
model.summary()

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 Ti (CNMeM is disabled, cuDNN 5110)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 640, 360)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 640, 360)  1792        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 320, 180)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 64, 320, 180)  256         maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [2]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(640, 360), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=180)

In [3]:
train_gen = image_aug(trainpath, batch_size=16)
valid_gen = batch_gen(validpath, batch_size=32)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [4]:
#model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=2, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/2
3214/3214 [==============================] - 221s - loss: 1.7070 - acc: 0.4543 - val_loss: 1.7060 - val_acc: 0.3961
Epoch 2/2
3214/3214 [==============================] - 210s - loss: 1.4992 - acc: 0.4997 - val_loss: 1.5699 - val_acc: 0.4547


In [5]:
model.save_weights(path+'/models/simpler_more_data_aug.h5')

In [6]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 217s - loss: 1.3368 - acc: 0.5348 - val_loss: 1.3381 - val_acc: 0.5595
Epoch 2/4
3214/3214 [==============================] - 209s - loss: 1.2586 - acc: 0.5647 - val_loss: 1.2854 - val_acc: 0.5702
Epoch 3/4
3214/3214 [==============================] - 210s - loss: 1.1925 - acc: 0.5912 - val_loss: 1.2417 - val_acc: 0.5861
Epoch 4/4
3214/3214 [==============================] - 211s - loss: 1.1536 - acc: 0.6033 - val_loss: 1.2387 - val_acc: 0.5560


In [7]:
model.save_weights(path+'/models/simpler_more_data_aug_2.h5')

In [8]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 213s - loss: 1.1397 - acc: 0.6114 - val_loss: 1.1128 - val_acc: 0.5808
Epoch 2/4
3214/3214 [==============================] - 202s - loss: 1.1102 - acc: 0.6161 - val_loss: 1.1921 - val_acc: 0.6341
Epoch 3/4
3214/3214 [==============================] - 203s - loss: 1.0759 - acc: 0.6170 - val_loss: 1.0390 - val_acc: 0.6181
Epoch 4/4
3214/3214 [==============================] - 204s - loss: 1.0561 - acc: 0.6276 - val_loss: 0.9030 - val_acc: 0.6536


In [ ]:
model.save_weights(path+'/models/simpler_more_data_aug_3.h5')

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 213s - loss: 1.0046 - acc: 0.6453 - val_loss: 0.9111 - val_acc: 0.6661
Epoch 2/4
3214/3214 [==============================] - 211s - loss: 0.9729 - acc: 0.6512 - val_loss: 0.9372 - val_acc: 0.6554
Epoch 3/4
3214/3214 [==============================] - 205s - loss: 0.9459 - acc: 0.6658 - val_loss: 0.9857 - val_acc: 0.6448
Epoch 4/4
3198/3214 [============================>.] - ETA: 0s - loss: 0.9138 - acc: 0.6807

In [ ]:
model.save_weights(path+'/models/simpler_more_data_aug_4.h5')

In [12]:
testpath = path + '/test_stg1'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)

Found 1000 images belonging to 1 classes.


In [13]:
predictions_one = model.predict_generator(test_gen, test_gen.nb_sample)

In [14]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/8, mx)

import pandas as pd
import os

classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_one = pd.DataFrame(do_clip(predictions_one, 0.92), columns=classes)
submission_one.insert(0, 'image', [os.path.basename(filename) for filename in test_gen.filenames])
submission_one.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.010000,0.010000,0.010000,0.010000,0.920000,0.010000,0.010000,0.010000
1,img_00007.jpg,0.409420,0.012094,0.010000,0.474855,0.043222,0.058334,0.010000,0.010000
2,img_00009.jpg,0.775162,0.016303,0.010000,0.010000,0.010000,0.200159,0.010000,0.010000
3,img_00018.jpg,0.374302,0.018517,0.010604,0.272065,0.010000,0.024913,0.010000,0.292219
4,img_00027.jpg,0.502995,0.010000,0.010000,0.010000,0.016810,0.047343,0.010818,0.414221


In [15]:
testpath = path + '/test_stg2'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)

Found 12153 images belonging to 1 classes.


In [16]:
predictions_two = model.predict_generator(test_gen, test_gen.nb_sample)

In [17]:
classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_two = pd.DataFrame(do_clip(predictions_two, 0.92), columns=classes)
submission_two.insert(0, 'image', ['test_stg2/' + os.path.basename(filename) for filename in test_gen.filenames])
submission_two.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_00001.jpg,0.036877,0.028955,0.01,0.338641,0.010000,0.159242,0.01,0.434715
1,test_stg2/image_00002.jpg,0.010000,0.010000,0.01,0.010000,0.920000,0.010000,0.01,0.010000
2,test_stg2/image_00003.jpg,0.328380,0.010000,0.01,0.010000,0.648453,0.021549,0.01,0.010000
3,test_stg2/image_00004.jpg,0.231518,0.045085,0.01,0.221323,0.354097,0.010000,0.01,0.139546
4,test_stg2/image_00005.jpg,0.010000,0.010000,0.01,0.010000,0.920000,0.010000,0.01,0.017282


In [18]:
submission = submission_one.append(submission_two, ignore_index=True)
submission.to_csv(os.path.join(path, 'stage_two_custom_clipping_augments.csv.gz'), index=False, compression='gzip')

Pseudo-labels?

In [19]:
sample_preds = submission_two.sample(frac=0.15)

In [26]:
import os
import shutil

pseudotrainpath = path + '/pseudotrain'
if not os.path.exists(pseudotrainpath):
    os.makedirs(pseudotrainpath)

for pred in sample_preds.itertuples(index=False):
    folder = sample_preds.columns[np.argmax(pred[1:]) + 1]
    if not os.path.exists(os.path.join(pseudotrainpath, folder)):
        os.makedirs(os.path.join(pseudotrainpath, folder))
    shutil.copyfile(path + '/test_stg2/unknown/' + os.path.basename(pred[0]), pseudotrainpath + '/' + folder + '/' + os.path.basename(pred[0]))

In [27]:
train_gen = image_aug(pseudotrainpath, batch_size=16)
valid_gen = batch_gen(validpath, batch_size=32)

Found 1823 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [28]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=2, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/2
1823/1823 [==============================] - 129s - loss: 0.8927 - acc: 0.7224 - val_loss: 1.4565 - val_acc: 0.5737
Epoch 2/2
1823/1823 [==============================] - 119s - loss: 0.7880 - acc: 0.7570 - val_loss: 1.5731 - val_acc: 0.5915


In [29]:
train_gen = image_aug(trainpath, batch_size=16)

Found 3214 images belonging to 8 classes.


In [30]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
2352/3214 [====================>.........] - ETA: 50s - loss: 1.0433 - acc: 0.6471

KeyboardInterrupt: 

In [ ]:
model.save_weights(path+'/models/simpler_more_data_aug_5.h5')

# More traditional design

In [2]:
path = 'data/fish'
trainpath = path + '/train'
samplepath = path + '/sample'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 640, 360)))
convblock(model, 3, 64)
convblock(model, 3, 128)
convblock(model, 3, 256)
convblock(model, 2, 128)
convblock(model, 2, 64)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))
model.summary()

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 Ti (CNMeM is disabled, cuDNN 5110)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 640, 360)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 640, 360)  1792        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 640, 360)  36928       convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)  (None, 64, 640, 360)  36928       convolution2d_2[0][0]            
___________________________________________________________________________________________

In [3]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(640, 360), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=180)

In [4]:
train_gen = image_aug(trainpath, batch_size=8)
valid_gen = batch_gen(validpath, batch_size=16)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [4]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=2, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/2
3214/3214 [==============================] - 534s - loss: 2.5300 - acc: 0.1826 - val_loss: 2.3963 - val_acc: 0.2842
Epoch 2/2
3214/3214 [==============================] - 533s - loss: 2.2398 - acc: 0.2505 - val_loss: 2.3369 - val_acc: 0.1901


In [5]:
model.save_weights(path+'/models/traditional.h5')

In [6]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 533s - loss: 2.1312 - acc: 0.2825 - val_loss: 1.9413 - val_acc: 0.2789
Epoch 2/4
3214/3214 [==============================] - 532s - loss: 2.0010 - acc: 0.3267 - val_loss: 1.9056 - val_acc: 0.3517
Epoch 3/4
3214/3214 [==============================] - 532s - loss: 1.9225 - acc: 0.3485 - val_loss: 1.7357 - val_acc: 0.3766
Epoch 4/4
3214/3214 [==============================] - 532s - loss: 1.8630 - acc: 0.3855 - val_loss: 1.9722 - val_acc: 0.3535


In [ ]:
model.save_weights(path+'/models/traditional_2.h5')

In [4]:
model.load_weights(path+'/models/traditional_2.h5')
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 530s - loss: 1.9487 - acc: 0.3398 - val_loss: 1.8662 - val_acc: 0.4174
Epoch 2/4
3214/3214 [==============================] - 529s - loss: 1.6554 - acc: 0.4269 - val_loss: 1.5940 - val_acc: 0.4618
Epoch 3/4
3214/3214 [==============================] - 528s - loss: 1.4859 - acc: 0.5028 - val_loss: 1.4409 - val_acc: 0.5098
Epoch 4/4
3214/3214 [==============================] - 531s - loss: 1.3968 - acc: 0.5233 - val_loss: 1.3789 - val_acc: 0.5222


In [ ]:
model.save_weights(path+'/models/traditional_3.h5')

In [5]:
model.load_weights(path+'/models/traditional_3.h5')
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 531s - loss: 1.2523 - acc: 0.5800 - val_loss: 1.2148 - val_acc: 0.5897
Epoch 2/3
3214/3214 [==============================] - 532s - loss: 1.2292 - acc: 0.5837 - val_loss: 1.1398 - val_acc: 0.6163
Epoch 3/3
3214/3214 [==============================] - 531s - loss: 1.1927 - acc: 0.5977 - val_loss: 1.1345 - val_acc: 0.6270


In [6]:
model.save_weights(path+'/models/traditional_4.h5')

In [7]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 526s - loss: 1.1466 - acc: 0.6133 - val_loss: 1.0929 - val_acc: 0.6217
Epoch 2/3
3214/3214 [==============================] - 528s - loss: 1.1420 - acc: 0.6123 - val_loss: 1.0561 - val_acc: 0.6501
Epoch 3/3
3214/3214 [==============================] - 529s - loss: 1.1075 - acc: 0.6226 - val_loss: 1.0842 - val_acc: 0.6767


In [8]:
model.save_weights(path+'/models/traditional_5.h5')

In [9]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 529s - loss: 1.0886 - acc: 0.6297 - val_loss: 1.0580 - val_acc: 0.6732
Epoch 2/3
3214/3214 [==============================] - 528s - loss: 1.0472 - acc: 0.6509 - val_loss: 0.9667 - val_acc: 0.6874
Epoch 3/3
3214/3214 [==============================] - 527s - loss: 1.0507 - acc: 0.6406 - val_loss: 0.9908 - val_acc: 0.6696


In [10]:
model.save_weights(path+'/models/traditional_6.h5')

In [12]:
testpath = path + '/test_stg1'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)
predictions_one = model.predict_generator(test_gen, test_gen.nb_sample)

Found 1000 images belonging to 1 classes.


In [13]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/8, mx)

import pandas as pd
import os

classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_one = pd.DataFrame(do_clip(predictions_one, 0.92), columns=classes)
submission_one.insert(0, 'image', [os.path.basename(filename) for filename in test_gen.filenames])
submission_one.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.311465,0.034179,0.198956,0.031402,0.282189,0.101234,0.027297,0.013277
1,img_00007.jpg,0.026094,0.142500,0.024009,0.508718,0.012018,0.064890,0.063603,0.158168
2,img_00009.jpg,0.205833,0.037878,0.012396,0.021625,0.513593,0.161785,0.015657,0.031234
3,img_00018.jpg,0.730616,0.011494,0.015934,0.042918,0.079380,0.026366,0.044917,0.048374
4,img_00027.jpg,0.562964,0.040978,0.018738,0.016989,0.013542,0.076207,0.198493,0.072088


In [14]:
testpath = path + '/test_stg2'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)

Found 12153 images belonging to 1 classes.


In [15]:
predictions_two = model.predict_generator(test_gen, test_gen.nb_sample)

In [16]:
classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_two = pd.DataFrame(do_clip(predictions_two, 0.92), columns=classes)
submission_two.insert(0, 'image', ['test_stg2/' + os.path.basename(filename) for filename in test_gen.filenames])
submission_two.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_00001.jpg,0.010216,0.018376,0.015547,0.024640,0.014235,0.555381,0.010554,0.351052
1,test_stg2/image_00002.jpg,0.010000,0.010000,0.010000,0.010000,0.920000,0.010000,0.010000,0.010000
2,test_stg2/image_00003.jpg,0.095582,0.010000,0.025407,0.010000,0.254244,0.120992,0.010000,0.487274
3,test_stg2/image_00004.jpg,0.039727,0.164521,0.053337,0.117737,0.019850,0.033516,0.054372,0.516940
4,test_stg2/image_00005.jpg,0.061531,0.089836,0.131566,0.090609,0.221638,0.062692,0.028299,0.313829


In [17]:
submission = submission_one.append(submission_two, ignore_index=True)
submission.to_csv(os.path.join(path, 'stage_two_more_traditional.csv.gz'), index=False, compression='gzip')

In [18]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 529s - loss: 1.0096 - acc: 0.6518 - val_loss: 0.9224 - val_acc: 0.7318
Epoch 2/3
3214/3214 [==============================] - 529s - loss: 1.0101 - acc: 0.6559 - val_loss: 1.0049 - val_acc: 0.6696
Epoch 3/3
3214/3214 [==============================] - 529s - loss: 0.9999 - acc: 0.6649 - val_loss: 0.9527 - val_acc: 0.6892


In [19]:
model.save_weights(path+'/models/traditional_7.h5')

In [20]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 529s - loss: 0.9844 - acc: 0.6500 - val_loss: 1.0392 - val_acc: 0.6732
Epoch 2/3
3214/3214 [==============================] - 528s - loss: 0.9828 - acc: 0.6699 - val_loss: 0.9233 - val_acc: 0.6856
Epoch 3/3
3214/3214 [==============================] - 529s - loss: 0.9474 - acc: 0.6854 - val_loss: 0.8566 - val_acc: 0.7300


In [21]:
model.save_weights(path+'/models/traditional_8.h5')

In [22]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 527s - loss: 0.9322 - acc: 0.6786 - val_loss: 0.8715 - val_acc: 0.7105
Epoch 2/4
3214/3214 [==============================] - 531s - loss: 0.9184 - acc: 0.6792 - val_loss: 0.9093 - val_acc: 0.6963
Epoch 3/4
3214/3214 [==============================] - 528s - loss: 0.8730 - acc: 0.7075 - val_loss: 0.8510 - val_acc: 0.7229
Epoch 4/4
3214/3214 [==============================] - 529s - loss: 0.9160 - acc: 0.6873 - val_loss: 0.8704 - val_acc: 0.7034


In [23]:
model.save_weights(path+'/models/traditional_9.h5')

In [24]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 529s - loss: 0.8670 - acc: 0.7032 - val_loss: 0.8556 - val_acc: 0.7140
Epoch 2/4
3214/3214 [==============================] - 529s - loss: 0.8646 - acc: 0.7044 - val_loss: 0.8269 - val_acc: 0.7318
Epoch 3/4
3214/3214 [==============================] - 528s - loss: 0.8609 - acc: 0.7153 - val_loss: 0.8664 - val_acc: 0.6980
Epoch 4/4
3214/3214 [==============================] - 529s - loss: 0.8273 - acc: 0.7166 - val_loss: 0.8090 - val_acc: 0.7282


In [25]:
model.save_weights(path+'/models/traditional_10.h5')

In [26]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 529s - loss: 0.8351 - acc: 0.7113 - val_loss: 0.8547 - val_acc: 0.7176
Epoch 2/6
3214/3214 [==============================] - 530s - loss: 0.8188 - acc: 0.7172 - val_loss: 0.7886 - val_acc: 0.7318
Epoch 3/6
3214/3214 [==============================] - 529s - loss: 0.8058 - acc: 0.7290 - val_loss: 0.9097 - val_acc: 0.6998
Epoch 4/6
3214/3214 [==============================] - 528s - loss: 0.8050 - acc: 0.7281 - val_loss: 0.8594 - val_acc: 0.7282
Epoch 5/6
3214/3214 [==============================] - 528s - loss: 0.7749 - acc: 0.7455 - val_loss: 0.7660 - val_acc: 0.7780
Epoch 6/6
3214/3214 [==============================] - 534s - loss: 0.7871 - acc: 0.7371 - val_loss: 0.8570 - val_acc: 0.7336


In [27]:
model.save_weights(path+'/models/traditional_11.h5')

In [11]:
def generate_submission(submission_path, clip=0.99):
    testpath = path + '/test_stg1'
    test_gen = batch_gen(testpath, class_mode=None, shuffle=False)
    predictions_one = model.predict_generator(test_gen, test_gen.nb_sample)
    
    def do_clip(arr, mx): return np.clip(arr, (1-mx)/8, mx)

    import pandas as pd
    import os

    classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
    submission_one = pd.DataFrame(do_clip(predictions_one, clip), columns=classes)
    submission_one.insert(0, 'image', [os.path.basename(filename) for filename in test_gen.filenames])
    
    testpath = path + '/test_stg2'
    test_gen = batch_gen(testpath, class_mode=None, shuffle=False)
    
    predictions_two = model.predict_generator(test_gen, test_gen.nb_sample)
    classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
    submission_two = pd.DataFrame(do_clip(predictions_two, 0.92), columns=classes)
    submission_two.insert(0, 'image', ['test_stg2/' + os.path.basename(filename) for filename in test_gen.filenames])
    
    submission = submission_one.append(submission_two, ignore_index=True)
    submission.to_csv(submission_path, index=False, compression='gzip')

In [29]:
generate_submission(path + '/more_training.csv.gz')

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


# Go deeper

In [3]:
path = 'data/fish'
trainpath = path + '/train'
samplepath = path + '/sample'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(BatchNormalization(axis=1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 640, 360)))
convblock(model, 2, 64)
convblock(model, 2, 128)
convblock(model, 3, 256)
convblock(model, 3, 512)
convblock(model, 3, 1024)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_19 (BatchNorm (None, 3, 640, 360)   12          batchnormalization_input_3[0][0] 
____________________________________________________________________________________________________
convolution2d_31 (Convolution2D) (None, 64, 640, 360)  1792        batchnormalization_19[0][0]      
____________________________________________________________________________________________________
convolution2d_32 (Convolution2D) (None, 64, 640, 360)  36928       convolution2d_31[0][0]           
____________________________________________________________________________________________________
maxpooling2d_13 (MaxPooling2D)   (None, 64, 320, 180)  0           convolution2d_32[0][0]           
___________________________________________________________________________________________

In [4]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(640, 360), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=180)

In [5]:
train_gen = image_aug(trainpath, batch_size=8)
valid_gen = batch_gen(validpath, batch_size=16)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [6]:
#model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=2, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/2
3214/3214 [==============================] - 589s - loss: 2.3192 - acc: 0.2931 - val_loss: 4.9259 - val_acc: 0.0568
Epoch 2/2
3214/3214 [==============================] - 587s - loss: 1.9946 - acc: 0.3696 - val_loss: 2.2069 - val_acc: 0.2984


In [7]:
model.save_weights(path+'/models/deeper_traditional.h5')

In [8]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 593s - loss: 1.8158 - acc: 0.4113 - val_loss: 2.1204 - val_acc: 0.3339
Epoch 2/6
3214/3214 [==============================] - 590s - loss: 1.6921 - acc: 0.4540 - val_loss: 1.8893 - val_acc: 0.4618
Epoch 3/6
3214/3214 [==============================] - 589s - loss: 1.5819 - acc: 0.4745 - val_loss: 1.6717 - val_acc: 0.4529
Epoch 4/6
3214/3214 [==============================] - 588s - loss: 1.5374 - acc: 0.4960 - val_loss: 1.7966 - val_acc: 0.4440
Epoch 5/6
3214/3214 [==============================] - 594s - loss: 1.4494 - acc: 0.5115 - val_loss: 1.7104 - val_acc: 0.4529
Epoch 6/6
3214/3214 [==============================] - 590s - loss: 1.4119 - acc: 0.5289 - val_loss: 1.4674 - val_acc: 0.5240


In [9]:
model.save_weights(path+'/models/deeper_traditional2.h5')

In [10]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 590s - loss: 1.3628 - acc: 0.5451 - val_loss: 1.4120 - val_acc: 0.5346
Epoch 2/6
3214/3214 [==============================] - 590s - loss: 1.3190 - acc: 0.5579 - val_loss: 1.3246 - val_acc: 0.5471
Epoch 3/6
3214/3214 [==============================] - 591s - loss: 1.2459 - acc: 0.5837 - val_loss: 1.4615 - val_acc: 0.4991
Epoch 4/6
3214/3214 [==============================] - 592s - loss: 1.2792 - acc: 0.5663 - val_loss: 1.3451 - val_acc: 0.5222
Epoch 5/6
3214/3214 [==============================] - 592s - loss: 1.1709 - acc: 0.5971 - val_loss: 1.2219 - val_acc: 0.6092
Epoch 6/6
3214/3214 [==============================] - 590s - loss: 1.1836 - acc: 0.5993 - val_loss: 1.1922 - val_acc: 0.5986


In [11]:
model.save_weights(path+'/models/deeper_traditional3.h5')

In [12]:
train_gen = image_aug(trainpath, batch_size=10)
valid_gen = batch_gen(validpath, batch_size=32)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [13]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 577s - loss: 1.0549 - acc: 0.6512 - val_loss: 1.0288 - val_acc: 0.6625
Epoch 2/6
3214/3214 [==============================] - 578s - loss: 1.0184 - acc: 0.6509 - val_loss: 1.0723 - val_acc: 0.6217
Epoch 3/6
3214/3214 [==============================] - 576s - loss: 0.9604 - acc: 0.6755 - val_loss: 0.9616 - val_acc: 0.6590
Epoch 4/6
3214/3214 [==============================] - 575s - loss: 0.9546 - acc: 0.6783 - val_loss: 0.9980 - val_acc: 0.6696
Epoch 5/6
3214/3214 [==============================] - 573s - loss: 0.9502 - acc: 0.6702 - val_loss: 0.9433 - val_acc: 0.6785
Epoch 6/6
3214/3214 [==============================] - 574s - loss: 0.9193 - acc: 0.6886 - val_loss: 0.9759 - val_acc: 0.6572


In [14]:
model.save_weights(path+'/models/deeper_traditional4.h5')

In [15]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 574s - loss: 0.9239 - acc: 0.6892 - val_loss: 0.9049 - val_acc: 0.7016
Epoch 2/6
3214/3214 [==============================] - 574s - loss: 0.8953 - acc: 0.6973 - val_loss: 0.9296 - val_acc: 0.6803
Epoch 3/6
3214/3214 [==============================] - 573s - loss: 0.8612 - acc: 0.7072 - val_loss: 0.7698 - val_acc: 0.7709
Epoch 4/6
3214/3214 [==============================] - 572s - loss: 0.8038 - acc: 0.7309 - val_loss: 0.8097 - val_acc: 0.6998
Epoch 5/6
3214/3214 [==============================] - 572s - loss: 0.8060 - acc: 0.7278 - val_loss: 0.9738 - val_acc: 0.7016
Epoch 6/6
3214/3214 [==============================] - 574s - loss: 0.8203 - acc: 0.7321 - val_loss: 1.0241 - val_acc: 0.7123


In [16]:
model.save_weights(path+'/models/deeper_traditional5.h5')

In [19]:
generate_submission(path + '/deeper.csv.gz')

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


In [20]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 574s - loss: 0.8200 - acc: 0.7172 - val_loss: 0.9886 - val_acc: 0.6838
Epoch 2/6
3214/3214 [==============================] - 574s - loss: 0.7507 - acc: 0.7486 - val_loss: 0.7503 - val_acc: 0.7265
Epoch 3/6
3214/3214 [==============================] - 573s - loss: 0.7406 - acc: 0.7536 - val_loss: 0.7902 - val_acc: 0.7496
Epoch 4/6
3214/3214 [==============================] - 572s - loss: 0.7329 - acc: 0.7498 - val_loss: 0.7836 - val_acc: 0.7353
Epoch 5/6
3214/3214 [==============================] - 572s - loss: 0.6839 - acc: 0.7654 - val_loss: 1.0554 - val_acc: 0.6643
Epoch 6/6
3214/3214 [==============================] - 572s - loss: 0.6997 - acc: 0.7642 - val_loss: 0.8176 - val_acc: 0.7300


In [21]:
model.save_weights(path+'/models/deeper_traditional6.h5')

In [22]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 572s - loss: 0.7033 - acc: 0.7589 - val_loss: 0.8590 - val_acc: 0.7087
Epoch 2/6
3214/3214 [==============================] - 572s - loss: 0.6549 - acc: 0.7797 - val_loss: 0.9136 - val_acc: 0.6821
Epoch 3/6
3214/3214 [==============================] - 574s - loss: 0.6375 - acc: 0.7866 - val_loss: 0.7236 - val_acc: 0.7602
Epoch 4/6
3214/3214 [==============================] - 573s - loss: 0.6848 - acc: 0.7735 - val_loss: 0.7624 - val_acc: 0.7460
Epoch 5/6
3214/3214 [==============================] - 572s - loss: 0.6658 - acc: 0.7738 - val_loss: 0.7151 - val_acc: 0.7425
Epoch 6/6
3214/3214 [==============================] - 572s - loss: 0.5887 - acc: 0.8002 - val_loss: 0.7230 - val_acc: 0.7940


In [23]:
model.save_weights(path+'/models/deeper_traditional7.h5')

In [24]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 572s - loss: 0.6035 - acc: 0.7946 - val_loss: 0.5210 - val_acc: 0.8242
Epoch 2/6
3214/3214 [==============================] - 572s - loss: 0.6121 - acc: 0.7943 - val_loss: 0.8845 - val_acc: 0.7638
Epoch 3/6
3214/3214 [==============================] - 573s - loss: 0.5552 - acc: 0.8055 - val_loss: 0.6608 - val_acc: 0.7957
Epoch 4/6
3214/3214 [==============================] - 573s - loss: 0.5458 - acc: 0.8143 - val_loss: 0.7389 - val_acc: 0.7762
Epoch 5/6
3214/3214 [==============================] - 573s - loss: 0.5472 - acc: 0.8090 - val_loss: 0.7206 - val_acc: 0.7851
Epoch 6/6
3214/3214 [==============================] - 573s - loss: 0.5454 - acc: 0.8195 - val_loss: 0.4756 - val_acc: 0.8703


In [25]:
model.save_weights(path+'/models/deeper_traditional8.h5')

In [26]:
model.optimizer.lr = 0.000001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 575s - loss: 0.5480 - acc: 0.8255 - val_loss: 0.5061 - val_acc: 0.8508
Epoch 2/6
3214/3214 [==============================] - 573s - loss: 0.5129 - acc: 0.8279 - val_loss: 0.6461 - val_acc: 0.7886
Epoch 3/6
3214/3214 [==============================] - 572s - loss: 0.5266 - acc: 0.8189 - val_loss: 0.6117 - val_acc: 0.8153
Epoch 4/6
3214/3214 [==============================] - 572s - loss: 0.4848 - acc: 0.8320 - val_loss: 0.6066 - val_acc: 0.8028
Epoch 5/6
3214/3214 [==============================] - 571s - loss: 0.4791 - acc: 0.8382 - val_loss: 0.5505 - val_acc: 0.8242
Epoch 6/6
3214/3214 [==============================] - 572s - loss: 0.4733 - acc: 0.8413 - val_loss: 0.5629 - val_acc: 0.8135


In [27]:
model.save_weights(path+'/models/deeper_traditional9.h5')

In [28]:
generate_submission(path + '/deeper2.csv.gz')

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


In [29]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 570s - loss: 0.4649 - acc: 0.8419 - val_loss: 0.5702 - val_acc: 0.8153
Epoch 2/6
3214/3214 [==============================] - 571s - loss: 0.4386 - acc: 0.8500 - val_loss: 0.4714 - val_acc: 0.8313
Epoch 3/6
3214/3214 [==============================] - 570s - loss: 0.4623 - acc: 0.8488 - val_loss: 0.4747 - val_acc: 0.8472
Epoch 4/6
3214/3214 [==============================] - 571s - loss: 0.4201 - acc: 0.8572 - val_loss: 0.4861 - val_acc: 0.8579
Epoch 5/6
3214/3214 [==============================] - 570s - loss: 0.4060 - acc: 0.8643 - val_loss: 0.4739 - val_acc: 0.8348
Epoch 6/6
3214/3214 [==============================] - 570s - loss: 0.4298 - acc: 0.8578 - val_loss: 0.4720 - val_acc: 0.8366


In [30]:
model.save_weights(path+'/models/deeper_traditional10.h5')

In [31]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 572s - loss: 0.4287 - acc: 0.8584 - val_loss: 0.5960 - val_acc: 0.8206
Epoch 2/6
3214/3214 [==============================] - 571s - loss: 0.4313 - acc: 0.8544 - val_loss: 0.4054 - val_acc: 0.8544
Epoch 3/6
3214/3214 [==============================] - 572s - loss: 0.4089 - acc: 0.8584 - val_loss: 0.4428 - val_acc: 0.8597
Epoch 4/6
3214/3214 [==============================] - 571s - loss: 0.3675 - acc: 0.8740 - val_loss: 0.4577 - val_acc: 0.8739
Epoch 5/6
3214/3214 [==============================] - 570s - loss: 0.4016 - acc: 0.8675 - val_loss: 0.4426 - val_acc: 0.8579
Epoch 6/6
3214/3214 [==============================] - 571s - loss: 0.3648 - acc: 0.8777 - val_loss: 0.3672 - val_acc: 0.8881


In [32]:
model.save_weights(path+'/models/deeper_traditional11.h5')

In [33]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 571s - loss: 0.3873 - acc: 0.8662 - val_loss: 0.3856 - val_acc: 0.8757
Epoch 2/6
3214/3214 [==============================] - 571s - loss: 0.3766 - acc: 0.8746 - val_loss: 0.3817 - val_acc: 0.8757
Epoch 3/6
3214/3214 [==============================] - 571s - loss: 0.3336 - acc: 0.8927 - val_loss: 0.4456 - val_acc: 0.8544
Epoch 4/6
3214/3214 [==============================] - 572s - loss: 0.3593 - acc: 0.8839 - val_loss: 0.3388 - val_acc: 0.9076
Epoch 5/6
3214/3214 [==============================] - 573s - loss: 0.3339 - acc: 0.8818 - val_loss: 0.3760 - val_acc: 0.8721
Epoch 6/6
3214/3214 [==============================] - 570s - loss: 0.3563 - acc: 0.8762 - val_loss: 0.4366 - val_acc: 0.8668


In [34]:
model.save_weights(path+'/models/deeper_traditional12.h5')

In [35]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 571s - loss: 0.3593 - acc: 0.8796 - val_loss: 0.3749 - val_acc: 0.8792
Epoch 2/6
3214/3214 [==============================] - 571s - loss: 0.3497 - acc: 0.8821 - val_loss: 0.4507 - val_acc: 0.8544
Epoch 3/6
3214/3214 [==============================] - 572s - loss: 0.3416 - acc: 0.8818 - val_loss: 0.3708 - val_acc: 0.8739
Epoch 4/6
3214/3214 [==============================] - 572s - loss: 0.3291 - acc: 0.8871 - val_loss: 0.3313 - val_acc: 0.8863
Epoch 5/6
3214/3214 [==============================] - 571s - loss: 0.2980 - acc: 0.8992 - val_loss: 0.4154 - val_acc: 0.8703
Epoch 6/6
3214/3214 [==============================] - 571s - loss: 0.3022 - acc: 0.8998 - val_loss: 0.2876 - val_acc: 0.9165


In [36]:
model.save_weights(path+'/models/deeper_traditional13.h5')

In [37]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 570s - loss: 0.3164 - acc: 0.8930 - val_loss: 0.4306 - val_acc: 0.8774
Epoch 2/6
3214/3214 [==============================] - 571s - loss: 0.3388 - acc: 0.8855 - val_loss: 0.4038 - val_acc: 0.8774
Epoch 3/6
3214/3214 [==============================] - 569s - loss: 0.2896 - acc: 0.9042 - val_loss: 0.4601 - val_acc: 0.8686
Epoch 4/6
3214/3214 [==============================] - 573s - loss: 0.2927 - acc: 0.9014 - val_loss: 0.2830 - val_acc: 0.9094
Epoch 5/6
3214/3214 [==============================] - 571s - loss: 0.2919 - acc: 0.9045 - val_loss: 0.3596 - val_acc: 0.8881
Epoch 6/6
3214/3214 [==============================] - 571s - loss: 0.2736 - acc: 0.9126 - val_loss: 0.3003 - val_acc: 0.9112


In [38]:
model.save_weights(path+'/models/deeper_traditional14.h5')

In [39]:
generate_submission(path + '/deeper3.csv.gz')

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


In [40]:
model.optimizer.lr = 0.0000001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 572s - loss: 0.2917 - acc: 0.9067 - val_loss: 0.2395 - val_acc: 0.9236
Epoch 2/6
3214/3214 [==============================] - 573s - loss: 0.2717 - acc: 0.9070 - val_loss: 0.3165 - val_acc: 0.9059
Epoch 3/6
3214/3214 [==============================] - 571s - loss: 0.2513 - acc: 0.9126 - val_loss: 0.2846 - val_acc: 0.9254
Epoch 4/6
3214/3214 [==============================] - 572s - loss: 0.2704 - acc: 0.9026 - val_loss: 0.3617 - val_acc: 0.8810
Epoch 5/6
3214/3214 [==============================] - 570s - loss: 0.2529 - acc: 0.9194 - val_loss: 0.2994 - val_acc: 0.9183
Epoch 6/6
3214/3214 [==============================] - 571s - loss: 0.2636 - acc: 0.9135 - val_loss: 0.3429 - val_acc: 0.9076


In [41]:
model.save_weights(path+'/models/deeper_traditional15.h5')

In [42]:
model.optimizer.lr = 0.0000001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 570s - loss: 0.2420 - acc: 0.9185 - val_loss: 0.2806 - val_acc: 0.9236
Epoch 2/6
3214/3214 [==============================] - 572s - loss: 0.2784 - acc: 0.9082 - val_loss: 0.3260 - val_acc: 0.9023
Epoch 3/6
3214/3214 [==============================] - 571s - loss: 0.2661 - acc: 0.9060 - val_loss: 0.3105 - val_acc: 0.9165
Epoch 4/6
3214/3214 [==============================] - 567s - loss: 0.2537 - acc: 0.9194 - val_loss: 0.2506 - val_acc: 0.9272
Epoch 5/6
3214/3214 [==============================] - 570s - loss: 0.2627 - acc: 0.9110 - val_loss: 0.3492 - val_acc: 0.9147
Epoch 6/6
3214/3214 [==============================] - 571s - loss: 0.2183 - acc: 0.9238 - val_loss: 0.3247 - val_acc: 0.9094


In [43]:
model.save_weights(path+'/models/deeper_traditional16.h5')

In [44]:
generate_submission(path + '/deeper4.csv.gz')

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


# Try another approach

In [1]:
path = 'data/fish'
trainpath = path + '/train'
samplepath = path + '/sample'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    #model.add(Dropout(0.1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 320, 180)))
convblock(model, 1, 64)
convblock(model, 1, 128)
# convblock(model, 3, 256)
# convblock(model, 3, 512)
# convblock(model, 3, 512)
# convblock(model, 3, 512)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(4096, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
model.add(Dense(8, activation='softmax'))
model.load_weights(path+'/models/simpler.h5')
model.summary()

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 Ti (CNMeM is disabled, cuDNN 5110)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 320, 180)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 320, 180)  1792        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 64, 320, 180)  256         convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 160, 90)   0           batchnormalization_2[0][0]       
___________________________________________________________________________________________

In [2]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(320, 180), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=180)

In [5]:
train_gen = batch_gen(samplepath, batch_size=45)
valid_gen = batch_gen(validpath, batch_size=64)

Found 173 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [23]:
model.optimizer.lr = 0.000001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
173/173 [==============================] - 18s - loss: 2.5351 - acc: 0.1618 - val_loss: 4.3884 - val_acc: 0.1758
Epoch 2/6
173/173 [==============================] - 15s - loss: 1.5864 - acc: 0.4624 - val_loss: 3.2970 - val_acc: 0.1829
Epoch 3/6
173/173 [==============================] - 15s - loss: 1.0412 - acc: 0.6590 - val_loss: 2.8770 - val_acc: 0.1812
Epoch 4/6
173/173 [==============================] - 15s - loss: 0.7192 - acc: 0.8092 - val_loss: 2.4520 - val_acc: 0.2309
Epoch 5/6
173/173 [==============================] - 15s - loss: 0.5196 - acc: 0.9133 - val_loss: 2.2656 - val_acc: 0.2167
Epoch 6/6
173/173 [==============================] - 15s - loss: 0.3966 - acc: 0.9364 - val_loss: 2.1282 - val_acc: 0.2327


In [24]:
model.optimizer.lr = 0.000001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
173/173 [==============================] - 18s - loss: 0.2451 - acc: 0.9884 - val_loss: 2.0637 - val_acc: 0.2380
Epoch 2/6
173/173 [==============================] - 15s - loss: 0.2469 - acc: 0.9942 - val_loss: 2.0067 - val_acc: 0.2753
Epoch 3/6
173/173 [==============================] - 15s - loss: 0.1815 - acc: 1.0000 - val_loss: 2.0175 - val_acc: 0.2416
Epoch 4/6
173/173 [==============================] - 15s - loss: 0.1539 - acc: 1.0000 - val_loss: 1.9991 - val_acc: 0.2487
Epoch 5/6
173/173 [==============================] - 15s - loss: 0.1351 - acc: 1.0000 - val_loss: 1.9975 - val_acc: 0.2416
Epoch 6/6
173/173 [==============================] - 15s - loss: 0.1010 - acc: 1.0000 - val_loss: 2.0191 - val_acc: 0.2274


In [27]:
model.save_weights(path+'/models/simpler.h5')

In [6]:
train_gen = batch_gen(trainpath, batch_size=64)

Found 3214 images belonging to 8 classes.


In [7]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 96s - loss: 0.8483 - acc: 0.7856 - val_loss: 1.6778 - val_acc: 0.4263
Epoch 2/6
3214/3214 [==============================] - 83s - loss: 0.2023 - acc: 0.9596 - val_loss: 1.7930 - val_acc: 0.3339
Epoch 3/6
3214/3214 [==============================] - 82s - loss: 0.0868 - acc: 0.9900 - val_loss: 1.7446 - val_acc: 0.3908
Epoch 4/6
3214/3214 [==============================] - 80s - loss: 0.0511 - acc: 0.9960 - val_loss: 1.3777 - val_acc: 0.5311
Epoch 5/6
3214/3214 [==============================] - 81s - loss: 0.0296 - acc: 0.9984 - val_loss: 1.0061 - val_acc: 0.7425
Epoch 6/6
3214/3214 [==============================] - 79s - loss: 0.0278 - acc: 0.9984 - val_loss: 0.5696 - val_acc: 0.8988


In [8]:
model.save_weights(path+'/models/simpler2.h5')

In [9]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 92s - loss: 0.0186 - acc: 0.9988 - val_loss: 0.4410 - val_acc: 0.9218
Epoch 2/6
3214/3214 [==============================] - 80s - loss: 0.0177 - acc: 0.9981 - val_loss: 0.2951 - val_acc: 0.9538
Epoch 3/6
3214/3214 [==============================] - 80s - loss: 0.0189 - acc: 0.9984 - val_loss: 0.2587 - val_acc: 0.9503
Epoch 4/6
3214/3214 [==============================] - 82s - loss: 0.0204 - acc: 0.9981 - val_loss: 0.2411 - val_acc: 0.9591
Epoch 5/6
3214/3214 [==============================] - 81s - loss: 0.0234 - acc: 0.9988 - val_loss: 0.1914 - val_acc: 0.9645
Epoch 6/6
3214/3214 [==============================] - 81s - loss: 0.0106 - acc: 0.9997 - val_loss: 0.1556 - val_acc: 0.9751


In [10]:
model.save_weights(path+'/models/simpler3.h5')

In [11]:
model.optimizer.lr = 0.0000001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 94s - loss: 0.0094 - acc: 0.9997 - val_loss: 0.2014 - val_acc: 0.9663
Epoch 2/6
3214/3214 [==============================] - 80s - loss: 0.0071 - acc: 0.9997 - val_loss: 0.1675 - val_acc: 0.9751
Epoch 3/6
3214/3214 [==============================] - 80s - loss: 0.0064 - acc: 0.9997 - val_loss: 0.1841 - val_acc: 0.9716
Epoch 4/6
3214/3214 [==============================] - 80s - loss: 0.0078 - acc: 0.9997 - val_loss: 0.1942 - val_acc: 0.9663
Epoch 5/6
3214/3214 [==============================] - 80s - loss: 0.0096 - acc: 0.9994 - val_loss: 0.1270 - val_acc: 0.9787
Epoch 6/6
3214/3214 [==============================] - 81s - loss: 0.0107 - acc: 0.9994 - val_loss: 0.1777 - val_acc: 0.9627


In [12]:
model.save_weights(path+'/models/simpler4.h5')

In [15]:
generate_submission(path + '/simpler1.csv.gz')

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


In [17]:
train_gen = image_aug(trainpath, batch_size=45)

Found 3214 images belonging to 8 classes.


In [18]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 97s - loss: 2.0832 - acc: 0.4032 - val_loss: 1.1814 - val_acc: 0.7211
Epoch 2/6
3214/3214 [==============================] - 85s - loss: 1.6370 - acc: 0.5047 - val_loss: 1.1020 - val_acc: 0.6750
Epoch 3/6
3214/3214 [==============================] - 85s - loss: 1.5210 - acc: 0.5376 - val_loss: 1.0018 - val_acc: 0.7496
Epoch 4/6
3214/3214 [==============================] - 85s - loss: 1.4574 - acc: 0.5498 - val_loss: 1.0609 - val_acc: 0.7389
Epoch 5/6
3214/3214 [==============================] - 85s - loss: 1.3866 - acc: 0.5840 - val_loss: 0.9602 - val_acc: 0.7407
Epoch 6/6
3214/3214 [==============================] - 85s - loss: 1.3443 - acc: 0.5965 - val_loss: 1.0225 - val_acc: 0.7513


In [19]:
model.save_weights(path+'/models/simpler5.h5')

In [20]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 95s - loss: 1.2571 - acc: 0.6245 - val_loss: 1.0092 - val_acc: 0.7478
Epoch 2/6
3214/3214 [==============================] - 87s - loss: 1.2445 - acc: 0.6210 - val_loss: 0.8864 - val_acc: 0.7922
Epoch 3/6
3214/3214 [==============================] - 86s - loss: 1.1933 - acc: 0.6381 - val_loss: 1.0119 - val_acc: 0.7496
Epoch 4/6
3214/3214 [==============================] - 86s - loss: 1.1831 - acc: 0.6428 - val_loss: 0.9609 - val_acc: 0.7691
Epoch 5/6
3214/3214 [==============================] - 88s - loss: 1.1363 - acc: 0.6577 - val_loss: 0.9328 - val_acc: 0.7709
Epoch 6/6
3214/3214 [==============================] - 88s - loss: 1.1286 - acc: 0.6565 - val_loss: 0.8816 - val_acc: 0.7744


In [21]:
def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0.10, channel_shift_range=0.10, horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=18)

In [22]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 96s - loss: 1.0829 - acc: 0.6783 - val_loss: 0.9194 - val_acc: 0.7851
Epoch 2/6
3214/3214 [==============================] - 86s - loss: 1.0596 - acc: 0.6686 - val_loss: 0.8807 - val_acc: 0.7798
Epoch 3/6
3214/3214 [==============================] - 86s - loss: 1.0580 - acc: 0.6792 - val_loss: 0.8309 - val_acc: 0.7904
Epoch 4/6
3214/3214 [==============================] - 86s - loss: 1.0080 - acc: 0.6910 - val_loss: 0.6645 - val_acc: 0.8348
Epoch 5/6
3214/3214 [==============================] - 85s - loss: 1.0154 - acc: 0.6882 - val_loss: 0.8041 - val_acc: 0.7940
Epoch 6/6
3214/3214 [==============================] - 85s - loss: 1.0082 - acc: 0.6920 - val_loss: 0.7766 - val_acc: 0.8206


In [23]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 97s - loss: 0.9717 - acc: 0.7019 - val_loss: 0.9038 - val_acc: 0.7549
Epoch 2/3
3214/3214 [==============================] - 87s - loss: 0.9756 - acc: 0.6982 - val_loss: 0.7637 - val_acc: 0.8295
Epoch 3/3
3214/3214 [==============================] - 87s - loss: 0.9608 - acc: 0.7001 - val_loss: 0.7720 - val_acc: 0.8135


In [30]:
def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0., channel_shift_range=0., horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=0.)

train_gen = image_aug(samplepath, batch_size=45)

Found 173 images belonging to 8 classes.


In [31]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
173/173 [==============================] - 18s - loss: 0.5996 - acc: 0.8324 - val_loss: 0.7987 - val_acc: 0.7726
Epoch 2/6
173/173 [==============================] - 15s - loss: 0.5384 - acc: 0.8844 - val_loss: 0.8057 - val_acc: 0.7549
Epoch 3/6
173/173 [==============================] - 15s - loss: 0.5402 - acc: 0.8671 - val_loss: 0.8270 - val_acc: 0.7549
Epoch 4/6
173/173 [==============================] - 15s - loss: 0.5673 - acc: 0.8497 - val_loss: 0.8518 - val_acc: 0.7389
Epoch 5/6
173/173 [==============================] - 15s - loss: 0.4859 - acc: 0.8728 - val_loss: 0.9145 - val_acc: 0.7087
Epoch 6/6
173/173 [==============================] - 15s - loss: 0.5030 - acc: 0.8902 - val_loss: 0.8805 - val_acc: 0.7194


In [32]:
train_gen = image_aug(trainpath, batch_size=50)

Found 3214 images belonging to 8 classes.


In [33]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 94s - loss: 0.8891 - acc: 0.7405 - val_loss: 0.4876 - val_acc: 0.8988
Epoch 2/6
3214/3214 [==============================] - 84s - loss: 0.6591 - acc: 0.8177 - val_loss: 0.5416 - val_acc: 0.8757
Epoch 3/6
3214/3214 [==============================] - 85s - loss: 0.5840 - acc: 0.8479 - val_loss: 0.4576 - val_acc: 0.9112
Epoch 4/6
3214/3214 [==============================] - 84s - loss: 0.5125 - acc: 0.8668 - val_loss: 0.5021 - val_acc: 0.9005
Epoch 5/6
3214/3214 [==============================] - 82s - loss: 0.4562 - acc: 0.8783 - val_loss: 0.3325 - val_acc: 0.9254
Epoch 6/6
3214/3214 [==============================] - 82s - loss: 0.4100 - acc: 0.9007 - val_loss: 0.3499 - val_acc: 0.9236


In [34]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 93s - loss: 0.3978 - acc: 0.8979 - val_loss: 0.3644 - val_acc: 0.9307
Epoch 2/4
3214/3214 [==============================] - 83s - loss: 0.3642 - acc: 0.9104 - val_loss: 0.3167 - val_acc: 0.9343
Epoch 3/4
3214/3214 [==============================] - 83s - loss: 0.3288 - acc: 0.9144 - val_loss: 0.3862 - val_acc: 0.9325
Epoch 4/4
3214/3214 [==============================] - 83s - loss: 0.3199 - acc: 0.9287 - val_loss: 0.3586 - val_acc: 0.9307


In [35]:
model.save_weights(path+'/models/simpler6.h5')

In [36]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 95s - loss: 0.2960 - acc: 0.9334 - val_loss: 0.3151 - val_acc: 0.9343
Epoch 2/4
3214/3214 [==============================] - 84s - loss: 0.2832 - acc: 0.9331 - val_loss: 0.3483 - val_acc: 0.9343
Epoch 3/4
3214/3214 [==============================] - 84s - loss: 0.2673 - acc: 0.9403 - val_loss: 0.2646 - val_acc: 0.9449
Epoch 4/4
3214/3214 [==============================] - 83s - loss: 0.2406 - acc: 0.9452 - val_loss: 0.2609 - val_acc: 0.9609


In [37]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 94s - loss: 0.2304 - acc: 0.9477 - val_loss: 0.3240 - val_acc: 0.9361
Epoch 2/4
3214/3214 [==============================] - 84s - loss: 0.2160 - acc: 0.9536 - val_loss: 0.2720 - val_acc: 0.9520
Epoch 3/4
3214/3214 [==============================] - 84s - loss: 0.2100 - acc: 0.9561 - val_loss: 0.2792 - val_acc: 0.9485
Epoch 4/4
3214/3214 [==============================] - 85s - loss: 0.2036 - acc: 0.9527 - val_loss: 0.2753 - val_acc: 0.9414


In [38]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=4, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/4
3214/3214 [==============================] - 94s - loss: 0.1962 - acc: 0.9564 - val_loss: 0.2355 - val_acc: 0.9627
Epoch 2/4
3214/3214 [==============================] - 84s - loss: 0.2094 - acc: 0.9546 - val_loss: 0.2369 - val_acc: 0.9520
Epoch 3/4
3214/3214 [==============================] - 83s - loss: 0.1808 - acc: 0.9605 - val_loss: 0.2405 - val_acc: 0.9627
Epoch 4/4
3214/3214 [==============================] - 83s - loss: 0.1747 - acc: 0.9655 - val_loss: 0.1840 - val_acc: 0.9698


In [40]:
model.save_weights(path+'/models/simpler7.h5')
generate_submission(path + '/simpler2.csv.gz')

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


In [43]:
def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0.1, channel_shift_range=0., horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=5)

train_gen = image_aug(trainpath, batch_size=55)

Found 3214 images belonging to 8 classes.


In [44]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=6, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/6
3214/3214 [==============================] - 93s - loss: 0.3975 - acc: 0.8958 - val_loss: 0.2591 - val_acc: 0.9467
Epoch 2/6
3214/3214 [==============================] - 81s - loss: 0.2913 - acc: 0.9328 - val_loss: 0.2467 - val_acc: 0.9591
Epoch 3/6
3214/3214 [==============================] - 82s - loss: 0.2507 - acc: 0.9499 - val_loss: 0.2626 - val_acc: 0.9591
Epoch 4/6
3214/3214 [==============================] - 82s - loss: 0.2488 - acc: 0.9459 - val_loss: 0.2479 - val_acc: 0.9538
Epoch 5/6
3214/3214 [==============================] - 81s - loss: 0.2195 - acc: 0.9555 - val_loss: 0.2598 - val_acc: 0.9467
Epoch 6/6
3214/3214 [==============================] - 82s - loss: 0.2204 - acc: 0.9518 - val_loss: 0.2344 - val_acc: 0.9520


In [45]:
model.save_weights(path+'/models/simpler8.h5')

In [46]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=10, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/10
3214/3214 [==============================] - 95s - loss: 0.2188 - acc: 0.9549 - val_loss: 0.2389 - val_acc: 0.9591
Epoch 2/10
3214/3214 [==============================] - 85s - loss: 0.2195 - acc: 0.9527 - val_loss: 0.2729 - val_acc: 0.9520
Epoch 3/10
3214/3214 [==============================] - 86s - loss: 0.2023 - acc: 0.9599 - val_loss: 0.2835 - val_acc: 0.9414
Epoch 4/10
3214/3214 [==============================] - 86s - loss: 0.2009 - acc: 0.9533 - val_loss: 0.2338 - val_acc: 0.9574
Epoch 5/10
3214/3214 [==============================] - 83s - loss: 0.1970 - acc: 0.9524 - val_loss: 0.2322 - val_acc: 0.9556
Epoch 6/10
3214/3214 [==============================] - 83s - loss: 0.1834 - acc: 0.9639 - val_loss: 0.2360 - val_acc: 0.9538
Epoch 7/10
3214/3214 [==============================] - 83s - loss: 0.1756 - acc: 0.9617 - val_loss: 0.2780 - val_acc: 0.9485
Epoch 8/10
3214/3214 [==============================] - 82s - loss: 0.1833 - acc: 0.9630 - val_loss: 0.2260 - val_acc:

In [47]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=10, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/10
3214/3214 [==============================] - 95s - loss: 0.1593 - acc: 0.9736 - val_loss: 0.2380 - val_acc: 0.9556
Epoch 2/10
3214/3214 [==============================] - 84s - loss: 0.1645 - acc: 0.9676 - val_loss: 0.2242 - val_acc: 0.9627
Epoch 3/10
3214/3214 [==============================] - 83s - loss: 0.1563 - acc: 0.9695 - val_loss: 0.2586 - val_acc: 0.9538
Epoch 4/10
3214/3214 [==============================] - 83s - loss: 0.1434 - acc: 0.9729 - val_loss: 0.2144 - val_acc: 0.9574
Epoch 5/10
3214/3214 [==============================] - 82s - loss: 0.1510 - acc: 0.9704 - val_loss: 0.1999 - val_acc: 0.9591
Epoch 6/10
3214/3214 [==============================] - 83s - loss: 0.1455 - acc: 0.9714 - val_loss: 0.2443 - val_acc: 0.9520
Epoch 7/10
3214/3214 [==============================] - 82s - loss: 0.1368 - acc: 0.9754 - val_loss: 0.2008 - val_acc: 0.9574
Epoch 8/10
3214/3214 [==============================] - 82s - loss: 0.1395 - acc: 0.9729 - val_loss: 0.2121 - val_acc:

In [48]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=10, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/10
3214/3214 [==============================] - 93s - loss: 0.1282 - acc: 0.9739 - val_loss: 0.2580 - val_acc: 0.9503
Epoch 2/10
3214/3214 [==============================] - 84s - loss: 0.1290 - acc: 0.9788 - val_loss: 0.1454 - val_acc: 0.9663
Epoch 3/10
3214/3214 [==============================] - 83s - loss: 0.1324 - acc: 0.9736 - val_loss: 0.2528 - val_acc: 0.9485
Epoch 4/10
3214/3214 [==============================] - 85s - loss: 0.1204 - acc: 0.9792 - val_loss: 0.1803 - val_acc: 0.9574
Epoch 5/10
3214/3214 [==============================] - 84s - loss: 0.1166 - acc: 0.9810 - val_loss: 0.2121 - val_acc: 0.9574
Epoch 6/10
3214/3214 [==============================] - 83s - loss: 0.1193 - acc: 0.9785 - val_loss: 0.2239 - val_acc: 0.9574
Epoch 7/10
3214/3214 [==============================] - 82s - loss: 0.1197 - acc: 0.9764 - val_loss: 0.2132 - val_acc: 0.9538
Epoch 8/10
3214/3214 [==============================] - 83s - loss: 0.1152 - acc: 0.9767 - val_loss: 0.1812 - val_acc:

In [49]:
model.save_weights(path+'/models/simpler9.h5')

In [50]:
generate_submission(path + '/simpler3.csv.gz', 0.91)

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


# Pump up the resolution

In [1]:
path = 'data/fish'
trainpath = path + '/train'
samplepath = path + '/sample'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    #model.add(Dropout(0.1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 640, 480)))
convblock(model, 1, 128)
convblock(model, 1, 128)
convblock(model, 1, 128)
# convblock(model, 3, 256)
# convblock(model, 3, 512)
# convblock(model, 3, 512)
# convblock(model, 3, 512)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(4096, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
model.add(Dense(8, activation='softmax'))
model.summary()

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 Ti (CNMeM is disabled, cuDNN 5110)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 640, 480)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 128, 640, 480) 3584        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 128, 640, 480) 512         convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 128, 320, 240) 0           batchnormalization_2[0][0]       
___________________________________________________________________________________________

In [2]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(640, 480), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0., channel_shift_range=0., horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=90.)

In [8]:
train_gen = image_aug(samplepath, batch_size=5)
valid_gen = batch_gen(validpath, batch_size=5)

Found 173 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [12]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=5, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/5
173/173 [==============================] - 58s - loss: 2.6688 - acc: 0.2370 - val_loss: 6.2084 - val_acc: 0.1137
Epoch 2/5
173/173 [==============================] - 58s - loss: 2.0922 - acc: 0.2890 - val_loss: 3.8607 - val_acc: 0.1421
Epoch 3/5
173/173 [==============================] - 57s - loss: 1.8699 - acc: 0.3757 - val_loss: 2.7940 - val_acc: 0.1261
Epoch 4/5
173/173 [==============================] - 57s - loss: 1.8440 - acc: 0.3179 - val_loss: 2.7391 - val_acc: 0.1901
Epoch 5/5
173/173 [==============================] - 57s - loss: 1.7205 - acc: 0.4220 - val_loss: 2.6560 - val_acc: 0.1794


In [5]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=5, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/5
173/173 [==============================] - 57s - loss: 2.5903 - acc: 0.2601 - val_loss: 12.8019 - val_acc: 0.1385
Epoch 2/5
173/173 [==============================] - 57s - loss: 1.9569 - acc: 0.2948 - val_loss: 12.1199 - val_acc: 0.1385
Epoch 3/5
173/173 [==============================] - 57s - loss: 1.8988 - acc: 0.3410 - val_loss: 10.1240 - val_acc: 0.1297
Epoch 4/5
173/173 [==============================] - 58s - loss: 1.6642 - acc: 0.4104 - val_loss: 5.6964 - val_acc: 0.1155
Epoch 5/5
173/173 [==============================] - 57s - loss: 1.8189 - acc: 0.3699 - val_loss: 5.5238 - val_acc: 0.1563


In [6]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=5, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/5
173/173 [==============================] - 58s - loss: 1.5014 - acc: 0.4162 - val_loss: 5.2720 - val_acc: 0.1385
Epoch 2/5
173/173 [==============================] - 57s - loss: 1.6148 - acc: 0.4046 - val_loss: 4.0422 - val_acc: 0.2078
Epoch 3/5
173/173 [==============================] - 57s - loss: 1.5852 - acc: 0.4335 - val_loss: 4.1763 - val_acc: 0.1794
Epoch 4/5
173/173 [==============================] - 58s - loss: 1.5163 - acc: 0.4740 - val_loss: 4.1396 - val_acc: 0.1599
Epoch 5/5
173/173 [==============================] - 58s - loss: 1.4520 - acc: 0.4277 - val_loss: 4.2580 - val_acc: 0.2256


In [9]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=5, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/5
173/173 [==============================] - 58s - loss: 1.6267 - acc: 0.4220 - val_loss: 4.1022 - val_acc: 0.2362
Epoch 2/5
173/173 [==============================] - 57s - loss: 1.7548 - acc: 0.3468 - val_loss: 6.3816 - val_acc: 0.1510
Epoch 3/5
173/173 [==============================] - 57s - loss: 1.6794 - acc: 0.4277 - val_loss: 4.7641 - val_acc: 0.1918
Epoch 4/5
173/173 [==============================] - 57s - loss: 1.5636 - acc: 0.3815 - val_loss: 4.7171 - val_acc: 0.1776
Epoch 5/5
173/173 [==============================] - 57s - loss: 1.5649 - acc: 0.4220 - val_loss: 4.4663 - val_acc: 0.2060


In [12]:
generate_submission(path + '/broader1.csv.gz', 0.95)

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


In [13]:
model.save_weights(path+'/models/broader1.h5')

In [14]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=15, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/15
173/173 [==============================] - 59s - loss: 1.7091 - acc: 0.3873 - val_loss: 4.4604 - val_acc: 0.1421
Epoch 2/15
173/173 [==============================] - 58s - loss: 1.5748 - acc: 0.4335 - val_loss: 4.2296 - val_acc: 0.2025
Epoch 3/15
173/173 [==============================] - 58s - loss: 1.5433 - acc: 0.4104 - val_loss: 4.1923 - val_acc: 0.2256
Epoch 4/15
173/173 [==============================] - 58s - loss: 1.4583 - acc: 0.4566 - val_loss: 4.1716 - val_acc: 0.2220
Epoch 5/15
173/173 [==============================] - 58s - loss: 1.5996 - acc: 0.4509 - val_loss: 4.6377 - val_acc: 0.1599
Epoch 6/15
173/173 [==============================] - 58s - loss: 1.5989 - acc: 0.4046 - val_loss: 4.1975 - val_acc: 0.2185
Epoch 7/15
173/173 [==============================] - 58s - loss: 1.4211 - acc: 0.4509 - val_loss: 4.1845 - val_acc: 0.2238
Epoch 8/15
173/173 [==============================] - 57s - loss: 1.5063 - acc: 0.3988 - val_loss: 4.0142 - val_acc: 0.2345
Epoch 9/

In [15]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=15, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/15
173/173 [==============================] - 59s - loss: 1.3631 - acc: 0.5087 - val_loss: 4.3654 - val_acc: 0.2558
Epoch 2/15
173/173 [==============================] - 61s - loss: 1.4254 - acc: 0.4509 - val_loss: 4.3932 - val_acc: 0.2291
Epoch 3/15
173/173 [==============================] - 59s - loss: 1.3600 - acc: 0.4798 - val_loss: 4.2725 - val_acc: 0.2575
Epoch 4/15
173/173 [==============================] - 58s - loss: 1.3481 - acc: 0.4855 - val_loss: 4.5690 - val_acc: 0.2540
Epoch 5/15
173/173 [==============================] - 58s - loss: 1.4350 - acc: 0.4682 - val_loss: 4.3131 - val_acc: 0.2433
Epoch 6/15
173/173 [==============================] - 59s - loss: 1.3098 - acc: 0.4682 - val_loss: 4.6375 - val_acc: 0.2202
Epoch 7/15
173/173 [==============================] - 58s - loss: 1.3477 - acc: 0.5318 - val_loss: 4.7081 - val_acc: 0.2522
Epoch 8/15
173/173 [==============================] - 58s - loss: 1.4562 - acc: 0.4624 - val_loss: 4.5374 - val_acc: 0.2202
Epoch 9/

In [16]:
model.save_weights(path+'/models/broader2.h5')

In [4]:
path = 'data/fish'
trainpath = path + '/train'
samplepath = path + '/sample'
validpath = path + '/valid'
models_path = path+'/models/'

from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

def convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    #model.add(Dropout(0.1))

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(3, 320, 240)))
convblock(model, 2, 64)
convblock(model, 2, 128)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(4096, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
model.add(Dense(8, activation='softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_13 (BatchNorm (None, 3, 320, 240)   12          batchnormalization_input_4[0][0] 
____________________________________________________________________________________________________
convolution2d_13 (Convolution2D) (None, 64, 320, 240)  1792        batchnormalization_13[0][0]      
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 320, 240)  36928       convolution2d_13[0][0]           
____________________________________________________________________________________________________
batchnormalization_14 (BatchNorm (None, 64, 320, 240)  256         convolution2d_14[0][0]           
___________________________________________________________________________________________

In [5]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True, rotation_range=0., zoom_range=0., channel_shift_range=0., horizontal_flip=False, width_shift_range=0., height_shift_range=0., batch_size=4):
    return ImageDataGenerator(rotation_range=rotation_range, zoom_range=zoom_range, channel_shift_range=channel_shift_range, horizontal_flip=horizontal_flip, width_shift_range=width_shift_range, height_shift_range=height_shift_range).flow_from_directory(path, target_size=(320, 240), batch_size=batch_size, class_mode=class_mode, shuffle=shuffle)

def image_aug(path, batch_size=4):
    return batch_gen(path, zoom_range=0., channel_shift_range=0., horizontal_flip=True, width_shift_range=0.05, height_shift_range=0.05, batch_size=batch_size, rotation_range=90.)

In [11]:
train_gen = image_aug(samplepath, batch_size=10)
valid_gen = batch_gen(validpath, batch_size=10)

Found 173 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [12]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=5, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/5
173/173 [==============================] - 22s - loss: 3.2400 - acc: 0.2197 - val_loss: 14.1427 - val_acc: 0.1226
Epoch 2/5
173/173 [==============================] - 21s - loss: 2.2597 - acc: 0.2948 - val_loss: 14.1713 - val_acc: 0.1208
Epoch 3/5
173/173 [==============================] - 21s - loss: 2.0067 - acc: 0.3468 - val_loss: 13.8613 - val_acc: 0.1332
Epoch 4/5
173/173 [==============================] - 21s - loss: 1.9956 - acc: 0.2832 - val_loss: 14.1339 - val_acc: 0.1208
Epoch 5/5
173/173 [==============================] - 21s - loss: 1.6476 - acc: 0.3931 - val_loss: 11.0235 - val_acc: 0.1208


In [13]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=10, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/10
173/173 [==============================] - 22s - loss: 1.8370 - acc: 0.3526 - val_loss: 12.2675 - val_acc: 0.1226
Epoch 2/10
173/173 [==============================] - 21s - loss: 1.7941 - acc: 0.3121 - val_loss: 11.4167 - val_acc: 0.1261
Epoch 3/10
173/173 [==============================] - 21s - loss: 1.6199 - acc: 0.4393 - val_loss: 9.0673 - val_acc: 0.1190
Epoch 4/10
173/173 [==============================] - 22s - loss: 1.7799 - acc: 0.4104 - val_loss: 7.2737 - val_acc: 0.2007
Epoch 5/10
173/173 [==============================] - 21s - loss: 1.8561 - acc: 0.3410 - val_loss: 5.7777 - val_acc: 0.1048
Epoch 6/10
173/173 [==============================] - 22s - loss: 1.7170 - acc: 0.3526 - val_loss: 5.1735 - val_acc: 0.1634
Epoch 7/10
173/173 [==============================] - 21s - loss: 1.6081 - acc: 0.4220 - val_loss: 6.0533 - val_acc: 0.1243
Epoch 8/10
173/173 [==============================] - 21s - loss: 1.7489 - acc: 0.3873 - val_loss: 5.0481 - val_acc: 0.1741
Epoch 

In [14]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=10, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/10
173/173 [==============================] - 22s - loss: 1.5337 - acc: 0.4624 - val_loss: 5.0228 - val_acc: 0.1954
Epoch 2/10
173/173 [==============================] - 21s - loss: 1.6869 - acc: 0.4220 - val_loss: 4.3332 - val_acc: 0.2309
Epoch 3/10
173/173 [==============================] - 21s - loss: 1.4602 - acc: 0.4624 - val_loss: 4.4523 - val_acc: 0.2469
Epoch 4/10
173/173 [==============================] - 21s - loss: 1.4697 - acc: 0.4971 - val_loss: 4.5017 - val_acc: 0.2274
Epoch 5/10
173/173 [==============================] - 21s - loss: 1.5564 - acc: 0.4104 - val_loss: 4.3711 - val_acc: 0.2345
Epoch 6/10
173/173 [==============================] - 22s - loss: 1.3781 - acc: 0.4971 - val_loss: 4.4008 - val_acc: 0.2185
Epoch 7/10
173/173 [==============================] - 21s - loss: 1.5176 - acc: 0.4855 - val_loss: 4.2485 - val_acc: 0.2718
Epoch 8/10
173/173 [==============================] - 21s - loss: 1.2613 - acc: 0.5376 - val_loss: 4.5509 - val_acc: 0.2700
Epoch 9/

In [16]:
train_gen = image_aug(samplepath, batch_size=20)
valid_gen = batch_gen(validpath, batch_size=20)
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=10, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Found 173 images belonging to 8 classes.
Found 563 images belonging to 8 classes.
Epoch 1/10
173/173 [==============================] - 22s - loss: 1.7784 - acc: 0.3815 - val_loss: 7.1835 - val_acc: 0.1155
Epoch 2/10
173/173 [==============================] - 20s - loss: 1.4852 - acc: 0.4451 - val_loss: 6.1015 - val_acc: 0.1226
Epoch 3/10
173/173 [==============================] - 20s - loss: 1.3721 - acc: 0.4798 - val_loss: 5.5007 - val_acc: 0.1297
Epoch 4/10
173/173 [==============================] - 20s - loss: 1.4050 - acc: 0.4798 - val_loss: 4.8847 - val_acc: 0.1439
Epoch 5/10
173/173 [==============================] - 20s - loss: 1.3718 - acc: 0.4509 - val_loss: 5.1518 - val_acc: 0.1243
Epoch 6/10
173/173 [==============================] - 20s - loss: 1.3046 - acc: 0.4971 - val_loss: 4.8243 - val_acc: 0.1705
Epoch 7/10
173/173 [==============================] - 20s - loss: 1.2740 - acc: 0.5549 - val_loss: 4.5763 - val_acc: 0.2060
Epoch 8/10
173/173 [==============================

In [17]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=10, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/10
173/173 [==============================] - 22s - loss: 1.2321 - acc: 0.5260 - val_loss: 4.5369 - val_acc: 0.2380
Epoch 2/10
173/173 [==============================] - 20s - loss: 1.1081 - acc: 0.6012 - val_loss: 5.0893 - val_acc: 0.1847
Epoch 3/10
173/173 [==============================] - 20s - loss: 1.2163 - acc: 0.4913 - val_loss: 5.5784 - val_acc: 0.1421
Epoch 4/10
173/173 [==============================] - 20s - loss: 1.1968 - acc: 0.5723 - val_loss: 5.2565 - val_acc: 0.1741
Epoch 5/10
173/173 [==============================] - 20s - loss: 1.2241 - acc: 0.5318 - val_loss: 4.9789 - val_acc: 0.2096
Epoch 6/10
173/173 [==============================] - 20s - loss: 1.2668 - acc: 0.5145 - val_loss: 4.9578 - val_acc: 0.1954
Epoch 7/10
173/173 [==============================] - 21s - loss: 1.1421 - acc: 0.5145 - val_loss: 5.1032 - val_acc: 0.2274
Epoch 8/10
173/173 [==============================] - 20s - loss: 1.1882 - acc: 0.5260 - val_loss: 4.7881 - val_acc: 0.2753
Epoch 9/

In [18]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=10, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/10
173/173 [==============================] - 22s - loss: 1.0961 - acc: 0.5838 - val_loss: 4.8412 - val_acc: 0.2522
Epoch 2/10
173/173 [==============================] - 22s - loss: 1.0295 - acc: 0.5896 - val_loss: 5.0344 - val_acc: 0.2664
Epoch 3/10
173/173 [==============================] - 21s - loss: 1.0568 - acc: 0.5838 - val_loss: 5.1990 - val_acc: 0.2842
Epoch 4/10
173/173 [==============================] - 21s - loss: 1.0125 - acc: 0.6069 - val_loss: 4.9887 - val_acc: 0.2806
Epoch 5/10
173/173 [==============================] - 21s - loss: 1.0479 - acc: 0.6185 - val_loss: 5.1566 - val_acc: 0.2753
Epoch 6/10
173/173 [==============================] - 21s - loss: 1.0575 - acc: 0.5549 - val_loss: 5.0430 - val_acc: 0.3020
Epoch 7/10
173/173 [==============================] - 21s - loss: 0.9853 - acc: 0.5954 - val_loss: 4.8895 - val_acc: 0.2931
Epoch 8/10
173/173 [==============================] - 21s - loss: 0.9772 - acc: 0.6301 - val_loss: 4.8925 - val_acc: 0.3197
Epoch 9/

In [19]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=20, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/20
173/173 [==============================] - 22s - loss: 0.9531 - acc: 0.6763 - val_loss: 4.8612 - val_acc: 0.2824
Epoch 2/20
173/173 [==============================] - 20s - loss: 0.8461 - acc: 0.7052 - val_loss: 5.0781 - val_acc: 0.3055
Epoch 3/20
173/173 [==============================] - 20s - loss: 1.0282 - acc: 0.6243 - val_loss: 5.2108 - val_acc: 0.2771
Epoch 4/20
173/173 [==============================] - 20s - loss: 0.8908 - acc: 0.6358 - val_loss: 5.4787 - val_acc: 0.2718
Epoch 5/20
173/173 [==============================] - 20s - loss: 0.9595 - acc: 0.6474 - val_loss: 5.0363 - val_acc: 0.3286
Epoch 6/20
173/173 [==============================] - 20s - loss: 0.9694 - acc: 0.6243 - val_loss: 4.8361 - val_acc: 0.3197
Epoch 7/20
173/173 [==============================] - 20s - loss: 0.8009 - acc: 0.6705 - val_loss: 5.4453 - val_acc: 0.2895
Epoch 8/20
173/173 [==============================] - 20s - loss: 0.9292 - acc: 0.5838 - val_loss: 4.9492 - val_acc: 0.2913
Epoch 9/

In [20]:
model.save_weights(path+'/models/sample.h5')